# `NEQ`: Collect bigrams corresponding to top adverbs

In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd

from source.utils.associate import POLAR_DIR, TOP_AM_DIR, adjust_assoc_columns
from source.utils.dataframes import update_assoc_index as update_index
from source.utils.general import (confirm_dir, print_iter, snake_to_camel,
                                  timestamp_today)
from am_notebooks import locate_bigram_am_paths, load_bigram_dfs, show_adv_bigrams, nb_show_table

ADV_FLOOR = 5000
K = 8

DATA_DATE = '2024-07-28'
TAG = 'NEQ'
TOP_AM_TAG_DIR = TOP_AM_DIR / TAG
confirm_dir(TOP_AM_TAG_DIR)

data_top = f'{TAG}-Top{K}'
OUT_DIR = TOP_AM_TAG_DIR / data_top
confirm_dir(OUT_DIR)

# for loading `polar/*/bigram/*` tables
bigram_floor = 25
mirror_floor = 5

## Set columns and diplay settings

In [2]:
FOCUS = ['f',
         'am_p1_given2', 'am_p1_given2_simple', 
         'conservative_log_ratio',
         'am_log_likelihood',
          'mutual_information',
         'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11', 'unexpected_f',
         'l1', 'l2', 
         'adv', 'adv_total', 'adj', 'adj_total']
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_columns', 9)
pd.set_option('display.width', 120)
pd.set_option("display.precision", 2)
pd.set_option("styler.format.precision", 2)
pd.set_option("styler.format.thousands", ",")
pd.set_option("display.float_format", '{:,.2f}'.format)

(MOVED the following to `./am_notebooks.py`)

In [3]:
# def force_ints(_df):
#     count_cols = _df.filter(regex=r'total|^[fN]').columns
#     _df[count_cols] = _df[count_cols].astype('int')
#     # _df[count_cols] = _df[:, count_cols].astype('int64')
#     # print(_df.dtypes.to_frame('dtypes'))
#     return _df


# def nb_show_table(df, n_dec: int = 2,
#                   adjust_columns: bool = True,
#                   outpath: Path = None,
#                   return_df: bool = False) -> None:
#     _df = df.copy()
#     try:
#         start_0 = _df.index.start == 0
#     except AttributeError:
#         pass
#     else:
#         _df.index.name = 'rank'
#         if start_0:
#             _df.index = _df.index + 1
#     if adjust_columns:
#         _df = adjust_assoc_columns(_df)
#     _df.columns = [f'`{c}`' for c in _df.columns]
#     _df.index = [f'**{r}**' for r in _df.index]
#     table = _df.to_markdown(floatfmt=f',.{n_dec}f', intfmt=',')
#     if outpath:
#         outpath.write_text(table)

#     print(f'\n{table}\n')
#     return (_df if return_df else None)
# from pprint import pprint
# from source.utils.dataframes import catify_hit_table as catify


# bigram_am_paths = {d.name:
              
#                   tuple(d.joinpath('bigram/extra').glob(
#                       f'*{TAG}*min{mirror_floor if d.name.endswith("mirror") else bigram_floor}x*extra.parq'))[0]
                  
#               for d in POLAR_DIR.iterdir()}

# pprint(bigram_am_paths)
# bigram_dfs = {n: catify(update_index(pd.read_parquet(p)), 
#                         reverse=True) 
#               for n, p in bigram_am_paths.items()}

# def show_adv_bigrams(sample_size, C, 
#                      bigram_dfs, 
#                      selector: str = 'dP1', 
#                      column_list: list = None) -> dict:
#     def _force_ints(_df):
#         count_cols = _df.filter(regex=r'total$|^[fN]').columns
#         _df.loc[:, count_cols] = _df.loc[:, count_cols].apply(
#             pd.to_numeric, downcast='unsigned')
#         return _df

#     def get_top_bigrams(bdf, adv, bigram_k):
#         bdf = _force_ints(bdf.loc[bdf.adv == adv, :])
#         top_by_metric = [bdf.nlargest(bigram_k * 2, m) for m in ['dP1', 'LRC']]
#         half_k = bigram_k // 2
#         adv_pat_bigrams = pd.concat(
#             [top_bigrams.head(half_k) for top_bigrams in top_by_metric]).drop_duplicates()
#         x = 0
#         while len(adv_pat_bigrams) < min(bigram_k, len(bdf)):
#             x += 1
#             next_ix = half_k + x
            
#             try:
#                 next_entries = [top_by_metric[0].iloc[[next_ix], :], 
#                             top_by_metric[1].iloc[[next_ix], :]]
#             except IndexError:
#                 print(f'All bigrams for {adv} retrieved.')
#                 break
#             else:
#                 adv_pat_bigrams = pd.concat((adv_pat_bigrams, 
#                                          *next_entries)).drop_duplicates()
#         return adv_pat_bigrams.head(bigram_k)

#     bigram_k = max(sample_size + 2, 10)
#     print(
#         f'## Top {bigram_k} "most negative" bigrams corresponding to top {sample_size} adverbs\n')
#     print(timestamp_today())
#     patterns = list(bigram_dfs.keys())
#     top_adverbs = C.index
#     bigram_samples = {adv: dict.fromkeys(
#         patterns + ['both', 'adj']) for adv in top_adverbs}
#     bigrams, adj = [], []

#     for rank, adv in enumerate(top_adverbs, start=1):
#         print(f'\n### {rank}. _{adv}_\n')
#         adj_for_adv = []
#         adv_top = None

#         for pat, bdf in bigram_dfs.items():
#             bdf = adjust_assoc_columns(
#                 bdf[[c for c in FOCUS + ['adj', 'adj_total', 'adv', 'adv_total'] if c in bdf.columns]])
#             bdf = bdf.loc[bdf.LRC >= 1, :]
#             adv_pat_bigrams = get_top_bigrams(bdf, adv, bigram_k)

#             if adv_pat_bigrams.empty:
#                 print(f'No bigrams found in loaded `{pat}` AM table.')
#             else:
#                 print(
#                     f'\n#### Top {len(adv_pat_bigrams)} `{pat}` "{adv}_*" bigrams (sorted by `{selector}`; `LRC > 1`)\n')
#                 column_list = column_list if column_list is not None else bdf.columns
#                 nb_show_table(adv_pat_bigrams.filter(column_list), n_dec=2)

#             adj_for_adv.extend(adv_pat_bigrams.adj.drop_duplicates().to_list())
#             bigram_samples[adv][pat] = adv_pat_bigrams
#             if adv_top is None: 
#                 adv_top = adv_pat_bigrams 
#             else:
#                 adv_top = pd.concat([df.fillna('') for df in (adv_top, adv_pat_bigrams)])

#         bigram_samples[adv]['adj'] = set(adj_for_adv)
#         bigrams.extend(adv_top.l2.drop_duplicates().to_list())
#         adj.extend(adj_for_adv)
#         bigram_samples[adv]['both'] = adv_top

#     bigram_samples['bigrams'] = set(bigrams)
#     bigram_samples['adj'] = set(adj)
#     return bigram_samples, bigram_k

## Load data

In [6]:
bigram_dfs = load_bigram_dfs(locate_bigram_am_paths(TAG, mirror_floor, bigram_floor))

try:
    combined_am_csv = tuple(OUT_DIR.glob(
                   f'{TAG}-Top{K}_NEG-ADV_combined-{ADV_FLOOR}*.{DATA_DATE or timestamp_today()}.csv'))[0]
except IndexError:
    combined_am_csv = tuple(TOP_AM_TAG_DIR.rglob(
        f'{TAG}-Top{K}_NEG-ADV_combined*.csv'))[0]
print(f'Loaded from: "{combined_am_csv}"')

C = adjust_assoc_columns(pd.read_csv(combined_am_csv, index_col='adv'))

main_cols_ordered = pd.concat((*[C.filter(like=m).columns.to_series() for m in ('LRC', 'P1', 'G2')],
                               *[C.filter(regex=f'^{f}_').columns.to_series() for f in ['f', 'f1', 'f2']])
                              ).to_list()
C

Loaded from: "/share/compling/projects/sanpi/results/top_AM/NEQ/NEQ-Top8/NEQ-Top8_NEG-ADV_combined-5000.2024-07-28.csv"


,key_SET,f_SET,dP1_SET,P1_SET,...,r_f_MIR,r_N_MIR,r_f1_MIR,r_f2_MIR
adv,,,,,,,,,
necessarily,NEGany~necessarily,42595,0.50,0.99,...,0.02,0.09,0.09,0.02
that,NEGany~that,164768,0.50,0.99,...,0.03,0.09,0.09,0.03
exactly,NEGany~exactly,43813,0.49,0.98,...,0.02,0.09,0.09,0.02
any,NEGany~any,15384,0.45,0.95,...,0.07,0.09,0.09,0.07
remotely,NEGany~remotely,5661,0.42,0.92,...,0.33,0.09,0.09,0.32
yet,NEGany~yet,51867,0.47,0.96,...,0.01,0.09,0.09,0.01
immediately,NEGany~immediately,56099,0.47,0.97,...,0.01,0.09,0.09,0.01
particularly,NEGany~particularly,55527,0.23,0.73,...,0.17,0.09,0.09,0.13
inherently,NEGany~inherently,6743,0.28,0.78,...,0.42,0.09,0.09,0.39


In [15]:
samples_dict, bigram_k = show_adv_bigrams(
    K, C, bigram_dfs,
    column_list=[
        'adj', 'adj_total',
        *pd.Series(main_cols_ordered).str.replace(
            r'mean_|_SET|_MIR', '', regex=True)
        .drop_duplicates().to_list(),
        # 't', 'MI'
    ]
)

## Top 10 "most negative" bigrams corresponding to top 8 adverbs

2024-07-28

### 1. _necessarily_


#### Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                   | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:----------------------------------|:-----------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~necessarily_indicative** | indicative |         2,313 |    6.29 |    0.50 |   1.00 | 1,925.89 | 1,389 | 3,173,660 |  1,389 |
| **NEGany~necessarily_easy**       | easy       |       108,923 |    5.67 |    0.50 |   1.00 | 1,260.28 |   909 | 3,173,660 |    909 |
| **NEGany~necessarily_new**        | new        |        21,538 |    4.74 |    0.50 |   1.00 |   668.24 |   482 | 3,173,660 |    482 |
| **NEGany~necessarily_surprising** | surprising |        18,776 |    4.23 |    0.50 |   1.00 |   471.36 |   340 | 3,173,660 |    340 |
| **N

## Top 10 "most negative" bigrams corresponding to top 8 adverbs

2024-07-28

### 1. _necessarily_


#### Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                   | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:----------------------------------|:-----------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~necessarily_indicative** | indicative |         2,313 |    6.29 |    0.50 |   1.00 | 1,925.89 | 1,389 | 3,173,660 |  1,389 |
| **NEGany~necessarily_easy**       | easy       |       108,923 |    5.67 |    0.50 |   1.00 | 1,260.28 |   909 | 3,173,660 |    909 |
| **NEGany~necessarily_new**        | new        |        21,538 |    4.74 |    0.50 |   1.00 |   668.24 |   482 | 3,173,660 |    482 |
| **NEGany~necessarily_surprising** | surprising |        18,776 |    4.23 |    0.50 |   1.00 |   471.36 |   340 | 3,173,660 |    340 |
| **NEGany~necessarily_enough**     | enough     |        27,603 |    3.93 |    0.50 |   1.00 |   386.79 |   279 | 3,173,660 |    279 |
| **NEGany~necessarily_bad**        | bad        |       119,509 |    6.31 |    0.50 |   1.00 | 2,814.04 | 2,059 | 3,173,660 |  2,062 |
| **NEGany~necessarily_true**       | true       |        34,967 |    6.16 |    0.50 |   1.00 | 4,330.74 | 3,232 | 3,173,660 |  3,245 |
| **NEGany~necessarily_better**     | better     |        50,827 |    6.07 |    0.50 |   1.00 | 2,564.81 | 1,887 | 3,173,660 |  1,891 |
| **NEGany~necessarily_aware**      | aware      |        28,973 |    3.48 |    0.50 |   1.00 |   285.59 |   206 | 3,173,660 |    206 |
| **NEGany~necessarily_related**    | related    |        14,260 |    5.14 |    0.50 |   1.00 | 1,013.51 |   741 | 3,173,660 |    742 |


#### Top 2 `mirror` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                              | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:-----------------------------|:--------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~necessarily_bad**   | bad     |         4,790 |    1.37 |    0.50 |   1.00 |  69.32 |    50 | 291,732 |     50 |
| **NEGmir~necessarily_wrong** | wrong   |         8,506 |    3.05 |    0.49 |   0.99 | 265.18 |   211 | 291,732 |    214 |


### 2. _that_


#### Top 10 `RBdirect` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                            | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |      `G2` |    `f` |      `f1` |   `f2` |
|:---------------------------|:-----------|--------------:|--------:|--------:|-------:|----------:|-------:|----------:|-------:|
| **NEGany~that_surprising** | surprising |        18,776 |    5.99 |    0.50 |   1.00 |  1,570.89 |  1,133 | 3,173,660 |  1,133 |
| **NEGany~that_unusual**    | unusual    |         7,412 |    5.77 |    0.50 |   1.00 |  1,354.57 |    977 | 3,173,660 |    977 |
| **NEGany~that_exciting**   | exciting   |        20,233 |    5.49 |    0.50 |   1.00 |  1,116.08 |    805 | 3,173,660 |    805 |
| **NEGany~that_uncommon**   | uncommon   |         3,165 |    5.49 |    0.50 |   1.00 |  1,111.92 |    802 | 3,173,660 |    802 |
| **NEGany~that_impressed**  | impressed  |        12,138 |    5.25 |    0.50 |   1.00 |    944.15 |    681 | 3,173,660 |    681 |
| **NEGany~that_hard**       | hard       |        45,061 |    7.68 |    0.50 |   1.00 | 13,602.42 |  9,948 | 3,173,660 |  9,963 |
| **NEGany~that_different**  | different  |        80,643 |    7.18 |    0.50 |   1.00 |  8,895.12 |  6,534 | 3,173,660 |  6,547 |
| **NEGany~that_great**      | great      |        45,359 |    7.18 |    0.50 |   1.00 | 14,908.90 | 11,032 | 3,173,660 | 11,065 |
| **NEGany~that_difficult**  | difficult  |        61,490 |    7.06 |    0.50 |   1.00 |  7,569.00 |  5,560 | 3,173,660 |  5,571 |
| **NEGany~that_big**        | big        |        42,912 |    6.47 |    0.50 |   1.00 |  8,332.69 |  6,244 | 3,173,660 |  6,273 |


#### Top 10 `mirror` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                            | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:---------------------------|:-----------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~that_close**      | close      |         4,831 |    1.67 |    0.50 |   1.00 |  83.19 |    60 | 291,732 |     60 |
| **NEGmir~that_happy**      | happy      |         5,463 |    1.03 |    0.50 |   1.00 |  56.84 |    41 | 291,732 |     41 |
| **NEGmir~that_popular**    | popular    |         2,841 |    1.54 |    0.48 |   0.98 |  81.14 |    65 | 291,732 |     66 |
| **NEGmir~that_simple**     | simple     |         7,465 |    3.67 |    0.48 |   0.98 | 580.44 |   474 | 291,732 |    483 |
| **NEGmir~that_difficult**  | difficult  |         4,854 |    1.16 |    0.48 |   0.98 |  63.56 |    52 | 291,732 |     53 |
| **NEGmir~that_easy**       | easy       |         7,749 |    3.23 |    0.47 |   0.97 | 512.43 |   450 | 291,732 |    465 |
| **NEGmir~that_great**      | great      |         2,123 |    2.71 |    0.46 |   0.96 | 312.65 |   286 | 291,732 |    298 |
| **NEGmir~that_good**       | good       |        13,423 |    2.65 |    0.44 |   0.94 | 441.70 |   447 | 291,732 |    476 |
| **NEGmir~that_big**        | big        |         3,134 |    2.08 |    0.47 |   0.97 | 132.98 |   113 | 291,732 |    116 |
| **NEGmir~that_interested** | interested |         2,877 |    1.26 |    0.47 |   0.97 |  70.93 |    62 | 291,732 |     64 |


### 3. _exactly_


#### Top 10 `RBdirect` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                               | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |      `G2` |   `f` |      `f1` |   `f2` |
|:------------------------------|:-----------|--------------:|--------:|--------:|-------:|----------:|------:|----------:|-------:|
| **NEGany~exactly_cheap**      | cheap      |         6,591 |    5.27 |    0.50 |   1.00 |    958.01 |   691 | 3,173,660 |    691 |
| **NEGany~exactly_surprising** | surprising |        18,776 |    4.61 |    0.50 |   1.00 |    610.01 |   440 | 3,173,660 |    440 |
| **NEGany~exactly_subtle**     | subtle     |         5,299 |    3.84 |    0.50 |   1.00 |    364.61 |   263 | 3,173,660 |    263 |
| **NEGany~exactly_fair**       | fair       |         6,964 |    3.83 |    0.50 |   1.00 |    360.45 |   260 | 3,173,660 |    260 |
| **NEGany~exactly_fun**        | fun        |        19,661 |    3.60 |    0.50 |   1.00 |    310.54 |   224 | 3,173,660 |    224 |
| **NEGany~exactly_sure**       | sure       |       134,139 |    7.46 |    0.50 |   1.00 | 11,991.61 | 8,794 | 3,173,660 |  8,810 |
| **NEGany~exactly_clear**      | clear      |        84,227 |    6.38 |    0.50 |   1.00 |  2,405.43 | 1,746 | 3,173,660 |  1,747 |
| **NEGany~exactly_new**        | new        |        21,538 |    6.03 |    0.50 |   1.00 |  1,885.86 | 1,371 | 3,173,660 |  1,372 |
| **NEGany~exactly_easy**       | easy       |       108,923 |    5.67 |    0.50 |   1.00 |  1,463.43 | 1,066 | 3,173,660 |  1,067 |
| **NEGany~exactly_hard**       | hard       |        45,061 |    3.46 |    0.50 |   1.00 |    281.43 |   203 | 3,173,660 |    203 |


#### Top 2 `mirror` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:-------------------------|:--------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~exactly_sure**  | sure    |         5,978 |    3.10 |    0.50 |   1.00 | 205.21 |   148 | 291,732 |    148 |
| **NEGmir~exactly_clear** | clear   |         3,321 |    1.16 |    0.48 |   0.98 |  63.56 |    52 | 291,732 |     53 |


### 4. _any_


#### Top 10 `RBdirect` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:-------------------------|:----------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~any_younger**   | younger   |         1,784 |    3.80 |    0.50 |   1.00 |   353.52 |   255 | 3,173,660 |    255 |
| **NEGany~any_nicer**     | nicer     |           642 |    2.30 |    0.50 |   1.00 |   133.09 |    96 | 3,173,660 |     96 |
| **NEGany~any_sweeter**   | sweeter   |           388 |    1.49 |    0.50 |   1.00 |    80.41 |    58 | 3,173,660 |     58 |
| **NEGany~any_happier**   | happier   |         2,004 |    4.66 |    0.49 |   0.99 | 1,085.12 |   828 | 3,173,660 |    834 |
| **NEGany~any_smarter**   | smarter   |           733 |    1.94 |    0.49 |   0.99 |   113.78 |    89 | 3,173,660 |     90 |
| **NEGany~any_easier**    | easier    |        12,877 |    4.42 |    0.48 |   0.98 | 1,946.26 | 1,594 | 3,173,660 |  1,625 |
| **NEGany~any_worse**     | worse     |        12,116 |    3.62 |    0.46 |   0.96 | 1,816.60 | 1,686 | 3,173,660 |  1,762 |
| **NEGany~any_better**    | better    |        50,827 |    3.59 |    0.44 |   0.94 | 4,753.39 | 4,719 | 3,173,660 |  5,004 |
| **NEGany~any_brighter**  | brighter  |           640 |    1.37 |    0.48 |   0.98 |    78.42 |    63 | 3,173,660 |     64 |
| **NEGany~any_different** | different |        80,643 |    3.03 |    0.44 |   0.94 |   905.82 |   902 | 3,173,660 |    957 |


#### Top 4 `mirror` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:-------------------------|:----------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~any_different** | different |         8,644 |    1.30 |    0.50 |   1.00 |  66.55 |    48 | 291,732 |     48 |
| **NEGmir~any_better**    | better    |         3,831 |    3.27 |    0.47 |   0.97 | 447.88 |   380 | 291,732 |    390 |
| **NEGmir~any_easier**    | easier    |           681 |    1.23 |    0.47 |   0.97 |  69.61 |    61 | 291,732 |     63 |
| **NEGmir~any_worse**     | worse     |         2,007 |    1.66 |    0.47 |   0.97 |  98.47 |    87 | 291,732 |     90 |


### 5. _remotely_


#### Top 10 `RBdirect` "remotely_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |      `f1` |   `f2` |
|:-------------------------------|:-----------|--------------:|--------:|--------:|-------:|-------:|------:|----------:|-------:|
| **NEGany~remotely_ready**      | ready      |        29,583 |    1.49 |    0.50 |   1.00 |  80.41 |    58 | 3,173,660 |     58 |
| **NEGany~remotely_enough**     | enough     |        27,603 |    1.13 |    0.50 |   1.00 |  65.16 |    47 | 3,173,660 |     47 |
| **NEGany~remotely_true**       | true       |        34,967 |    3.53 |    0.50 |   1.00 | 334.93 |   250 | 3,173,660 |    251 |
| **NEGany~remotely_surprising** | surprising |        18,776 |    1.66 |    0.49 |   0.99 |  94.71 |    75 | 3,173,660 |     76 |
| **NEGany~remotely_funny**      | funny      |        14,992 |    2.16 |    0.47 |   0.97 | 159.09 |   137 | 3,173,660 |    141 |
| **NEGany~remotely_close**      | close      |        46,485 |    2.98 |    0.45 |   0.95 | 711.52 |   694 | 3,173,660 |    733 |
| **NEGany~remotely_interested** | interested |        34,543 |    1.99 |    0.41 |   0.91 | 278.69 |   330 | 3,173,660 |    364 |
| **NEGany~remotely_comparable** | comparable |         2,401 |    1.62 |    0.44 |   0.94 | 119.34 |   118 | 3,173,660 |    125 |
| **NEGany~remotely_similar**    | similar    |        11,088 |    1.39 |    0.40 |   0.90 | 123.97 |   152 | 3,173,660 |    169 |
| **NEGany~remotely_related**    | related    |        14,260 |    1.33 |    0.40 |   0.90 | 116.95 |   146 | 3,173,660 |    163 |


#### Top 3 `mirror` "remotely_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:-------------------------------|:-----------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~remotely_comparable** | comparable |           158 |    1.15 |    0.50 |   1.00 |  61.00 |    44 | 291,732 |     44 |
| **NEGmir~remotely_true**       | true       |         2,850 |    1.43 |    0.48 |   0.98 |  75.72 |    61 | 291,732 |     62 |
| **NEGmir~remotely_close**      | close      |         4,831 |    2.58 |    0.46 |   0.96 | 244.21 |   218 | 291,732 |    226 |


### 6. _yet_


#### Top 10 `RBdirect` "yet_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `P1` |      `G2` |    `f` |      `f1` |   `f2` |
|:-------------------------|:----------|--------------:|--------:|--------:|-------:|----------:|-------:|----------:|-------:|
| **NEGany~yet_clear**     | clear     |        84,227 |    8.66 |    0.50 |   1.00 | 14,392.25 | 10,406 | 3,173,660 | 10,409 |
| **NEGany~yet_certain**   | certain   |        11,334 |    5.60 |    0.50 |   1.00 |  1,200.66 |    866 | 3,173,660 |    866 |
| **NEGany~yet_ready**     | ready     |        29,583 |    8.06 |    0.50 |   1.00 | 10,344.81 |  7,501 | 3,173,660 |  7,505 |
| **NEGany~yet_final**     | final     |         1,213 |    5.16 |    0.50 |   1.00 |    887.30 |    640 | 3,173,660 |    640 |
| **NEGany~yet_public**    | public    |         2,656 |    4.69 |    0.50 |   1.00 |    647.44 |    467 | 3,173,660 |    467 |
| **NEGany~yet_complete**  | complete  |         8,415 |    6.70 |    0.50 |   1.00 |  2,998.60 |  2,174 | 3,173,660 |  2,175 |
| **NEGany~yet_available** | available |        82,956 |    6.66 |    0.50 |   1.00 |  9,950.03 |  7,430 | 3,173,660 |  7,461 |
| **NEGany~yet_sure**      | sure      |       134,139 |    6.13 |    0.50 |   1.00 |  2,689.26 |  1,977 | 3,173,660 |  1,981 |
| **NEGany~yet_dead**      | dead      |         6,348 |    4.47 |    0.50 |   1.00 |    555.93 |    401 | 3,173,660 |    401 |
| **NEGany~yet_able**      | able      |        23,355 |    5.44 |    0.50 |   1.00 |  1,764.46 |  1,315 | 3,173,660 |  1,320 |

No bigrams found in loaded `mirror` AM table.

### 7. _immediately_


#### Top 10 `RBdirect` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                   | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |      `G2` |    `f` |      `f1` |   `f2` |
|:----------------------------------|:-----------|--------------:|--------:|--------:|-------:|----------:|-------:|----------:|-------:|
| **NEGany~immediately_sure**       | sure       |       134,139 |    2.87 |    0.50 |   1.00 |    191.31 |    138 | 3,173,660 |    138 |
| **NEGany~immediately_reachable**  | reachable  |           350 |    2.50 |    0.50 |   1.00 |    151.11 |    109 | 3,173,660 |    109 |
| **NEGany~immediately_certain**    | certain    |        11,334 |    1.80 |    0.50 |   1.00 |     97.04 |     70 | 3,173,660 |     70 |
| **NEGany~immediately_clear**      | clear      |        84,227 |    7.55 |    0.50 |   1.00 | 33,058.44 | 24,416 | 3,173,660 | 24,488 |
| **NEGany~immediately_possible**   | possible   |        30,446 |    5.40 |    0.50 |   1.00 |  1,360.38 |  1,000 | 3,173,660 |  1,002 |
| **NEGany~immediately_available**  | available  |        82,956 |    5.34 |    0.48 |   0.98 | 25,870.14 | 21,078 | 3,173,660 | 21,477 |
| **NEGany~immediately_obvious**    | obvious    |        22,651 |    3.88 |    0.46 |   0.96 |  2,481.50 |  2,238 | 3,173,660 |  2,325 |
| **NEGany~immediately_able**       | able       |        23,355 |    3.66 |    0.48 |   0.98 |    746.39 |    626 | 3,173,660 |    641 |
| **NEGany~immediately_successful** | successful |        31,460 |    2.87 |    0.47 |   0.97 |    333.73 |    290 | 3,173,660 |    299 |
| **NEGany~immediately_apparent**   | apparent   |         9,798 |    3.30 |    0.44 |   0.94 |  2,001.83 |  2,015 | 3,173,660 |  2,143 |


#### Top 1 `mirror` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                  | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:---------------------------------|:----------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~immediately_available** | available |         3,079 |    1.34 |    0.38 |   0.88 | 120.41 |   162 | 291,732 |    184 |


### 8. _particularly_


#### Top 10 `RBdirect` "particularly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                    | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:-----------------------------------|:-----------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **COM~particularly_acute**         | acute      |         1,038 |    2.84 |    0.50 |   1.00 |   187.16 |   135 | 3,173,552 |    135 |
| **NEGany~particularly_wrong**      | wrong      |        21,332 |    3.56 |    0.50 |   1.00 |   302.22 |   218 | 3,173,660 |    218 |
| **NEGany~particularly_athletic**   | athletic   |         1,772 |    2.49 |    0.50 |   1.00 |   149.72 |   108 | 3,173,660 |    108 |
| **NEGany~particularly_likeable**   | likeable   |           861 |    2.46 |    0.50 |   1.00 |   146.95 |   106 | 3,173,660 |    106 |
| **NEGany~particularly_radical**    | radical    |         2,637 |    1.99 |    0.50 |   1.00 |   109.52 |    79 | 3,173,660 |     79 |
| **NEGany~particularly_new**        | new        |        21,538 |    4.61 |    0.49 |   0.99 |   982.49 |   747 | 3,173,660 |    752 |
| **NEGany~particularly_religious**  | religious  |         3,507 |    4.52 |    0.50 |   1.00 |   659.41 |   485 | 3,173,660 |    486 |
| **NEGany~particularly_surprising** | surprising |        18,776 |    3.93 |    0.47 |   0.97 | 1,260.26 | 1,069 | 3,173,660 |  1,097 |
| **NEGany~particularly_original**   | original   |         4,693 |    3.64 |    0.49 |   0.99 |   460.59 |   360 | 3,173,660 |    364 |
| **NEGany~particularly_flashy**     | flashy     |         1,732 |    1.46 |    0.50 |   1.00 |    79.02 |    57 | 3,173,660 |     57 |


#### Top 10 `mirror` "particularly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                     | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:------------------------------------|:------------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~particularly_surprising**  | surprising  |         1,248 |    3.27 |    0.50 |   1.00 | 230.18 |   166 | 291,732 |    166 |
| **NEGmir~particularly_original**    | original    |           715 |    2.33 |    0.50 |   1.00 | 124.78 |    90 | 291,732 |     90 |
| **NEGmir~particularly_novel**       | novel       |           179 |    1.50 |    0.50 |   1.00 |  74.87 |    54 | 291,732 |     54 |
| **NEGmir~particularly_religious**   | religious   |           337 |    1.47 |    0.50 |   1.00 |  73.48 |    53 | 291,732 |     53 |
| **NEGmir~particularly_innovative**  | innovative  |           675 |    1.26 |    0.50 |   1.00 |  65.16 |    47 | 291,732 |     47 |
| **NEGmir~particularly_new**         | new         |         4,300 |    4.35 |    0.50 |   1.00 | 547.73 |   404 | 291,732 |    405 |
| **NEGmir~particularly_wrong**       | wrong       |         8,506 |    3.39 |    0.50 |   1.00 | 282.64 |   212 | 291,732 |    213 |
| **NEGmir~particularly_good**        | good        |        13,423 |    3.24 |    0.47 |   0.97 | 455.35 |   390 | 291,732 |    401 |
| **NEGmir~particularly_unusual**     | unusual     |           933 |    2.72 |    0.48 |   0.98 | 209.60 |   170 | 291,732 |    173 |
| **NEGmir~particularly_comfortable** | comfortable |         1,888 |    1.15 |    0.50 |   1.00 |  61.00 |    44 | 291,732 |     44 |


### 9. _inherently_


#### Top 6 `RBdirect` "inherently_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                               | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:------------------------------|:--------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~inherently_illegal** | illegal |         3,580 |    1.26 |    0.48 |   0.98 |    73.01 |    59 | 3,173,660 |     60 |
| **NEGany~inherently_bad**     | bad     |       119,509 |    3.87 |    0.48 |   0.98 |   953.05 |   794 | 3,173,660 |    812 |
| **NEGany~inherently_wrong**   | wrong   |        21,332 |    4.25 |    0.48 |   0.98 | 1,956.12 | 1,639 | 3,173,660 |  1,678 |
| **NEGany~inherently_evil**    | evil    |         3,171 |    2.12 |    0.41 |   0.91 |   312.23 |   358 | 3,173,660 |    392 |
| **NEGany~inherently_better**  | better  |        50,827 |    1.46 |    0.41 |   0.91 |   124.46 |   144 | 3,173,660 |    158 |
| **NEGany~inherently_good**    | good    |       201,244 |    1.46 |    0.36 |   0.86 |   189.85 |   283 | 3,173,660 |    329 |


#### Top 2 `mirror` "inherently_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                             | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |
|:----------------------------|:--------|--------------:|--------:|--------:|-------:|---------:|------:|--------:|-------:|
| **NEGmir~inherently_wrong** | wrong   |         8,506 |    3.78 |    0.46 |   0.96 | 1,685.02 | 1,513 | 291,732 |  1,571 |
| **NEGmir~inherently_bad**   | bad     |         4,790 |    1.83 |    0.44 |   0.94 |   144.52 |   148 | 291,732 |    158 |


### 10. _terribly_


#### Top 10 `RBdirect` "terribly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                 | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:--------------------------------|:------------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~terribly_surprising**  | surprising  |        18,776 |    5.73 |    0.50 |   1.00 | 1,315.75 |   949 | 3,173,660 |    949 |
| **NEGany~terribly_popular**     | popular     |        51,120 |    2.99 |    0.50 |   1.00 |   206.56 |   149 | 3,173,660 |    149 |
| **NEGany~terribly_unusual**     | unusual     |         7,412 |    2.96 |    0.50 |   1.00 |   202.40 |   146 | 3,173,660 |    146 |
| **NEGany~terribly_comfortable** | comfortable |        23,908 |    2.77 |    0.50 |   1.00 |   178.84 |   129 | 3,173,660 |    129 |
| **NEGany~terribly_bright**      | bright      |         8,623 |    2.61 |    0.50 |   1.00 |   162.20 |   117 | 3,173,660 |    117 |
| **NEGany~terribly_interested**  | interested  |        34,543 |    3.98 |    0.49 |   0.99 |   624.89 |   486 | 3,173,660 |    491 |
| **NEGany~terribly_different**   | different   |        80,643 |    3.93 |    0.49 |   0.99 |   485.33 |   366 | 3,173,660 |    368 |
| **NEGany~terribly_surprised**   | surprised   |        10,157 |    3.30 |    0.49 |   0.99 |   361.19 |   287 | 3,173,660 |    291 |
| **NEGany~terribly_exciting**    | exciting    |        20,233 |    3.28 |    0.48 |   0.98 |   456.39 |   382 | 3,173,660 |    391 |
| **NEGany~terribly_common**      | common      |        34,450 |    2.45 |    0.50 |   1.00 |   145.56 |   105 | 3,173,660 |    105 |


#### Top 5 `mirror` "terribly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                 | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:--------------------------------|:------------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~terribly_surprising**  | surprising  |         1,248 |    1.85 |    0.50 |   1.00 |  92.89 |    67 | 291,732 |     67 |
| **NEGmir~terribly_original**    | original    |           715 |    1.19 |    0.50 |   1.00 |  62.39 |    45 | 291,732 |     45 |
| **NEGmir~terribly_new**         | new         |         4,300 |    1.64 |    0.49 |   0.99 |  86.57 |    69 | 291,732 |     70 |
| **NEGmir~terribly_interesting** | interesting |         3,863 |    1.29 |    0.48 |   0.98 |  68.96 |    56 | 291,732 |     57 |
| **POS~terribly_wrong**          | wrong       |         8,506 |    1.06 |    0.30 |   0.80 | 149.75 |   319 | 291,729 |    401 |


### 11. _ever_


#### Top 10 `RBdirect` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                        | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |      `f1` |   `f2` |
|:-----------------------|:--------|--------------:|--------:|--------:|-------:|-------:|------:|----------:|-------:|
| **COM~ever_larger**    | larger  |         7,453 |    2.88 |    0.50 |   1.00 | 192.71 |   139 | 3,173,552 |    139 |
| **NEGany~ever_boring** | boring  |         3,840 |    1.84 |    0.50 |   1.00 |  99.82 |    72 | 3,173,660 |     72 |
| **NEGany~ever_simple** | simple  |        46,867 |    3.28 |    0.50 |   1.00 | 281.20 |   211 | 3,173,660 |    212 |
| **COM~ever_greater**   | greater |         6,949 |    3.09 |    0.49 |   0.99 | 246.80 |   186 | 3,173,552 |    187 |
| **COM~ever_closer**    | closer  |         3,686 |    3.52 |    0.49 |   0.99 | 365.82 |   279 | 3,173,552 |    281 |
| **NEGany~ever_easy**   | easy    |       108,923 |    3.53 |    0.48 |   0.98 | 525.98 |   429 | 3,173,660 |    437 |
| **COM~ever_deeper**    | deeper  |         1,768 |    1.31 |    0.48 |   0.98 |  75.72 |    61 | 3,173,552 |     62 |
| **COM~ever_higher**    | higher  |        12,992 |    2.52 |    0.49 |   0.99 | 168.50 |   129 | 3,173,552 |    130 |
| **NEGany~ever_good**   | good    |       201,244 |    2.52 |    0.45 |   0.95 | 337.56 |   331 | 3,173,660 |    350 |
| **COM~ever_mindful**   | mindful |           784 |    1.04 |    0.48 |   0.98 |  63.56 |    52 | 3,173,552 |     53 |


#### Top 10 `mirror` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                         | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:------------------------|:--------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~ever_perfect** | perfect |         1,303 |    3.60 |    0.50 |   1.00 | 285.65 |   206 | 291,732 |    206 |
| **NEGmir~ever_simple**  | simple  |         7,465 |    3.60 |    0.50 |   1.00 | 285.65 |   206 | 291,732 |    206 |
| **NEGmir~ever_enough**  | enough  |         1,326 |    3.09 |    0.50 |   1.00 | 203.83 |   147 | 291,732 |    147 |
| **NEGmir~ever_certain** | certain |         1,276 |    3.04 |    0.50 |   1.00 | 198.28 |   143 | 291,732 |    143 |
| **NEGmir~ever_wrong**   | wrong   |         8,506 |    2.52 |    0.50 |   1.00 | 141.42 |   102 | 291,732 |    102 |
| **NEGmir~ever_easy**    | easy    |         7,749 |    4.21 |    0.50 |   1.00 | 497.96 |   368 | 291,732 |    369 |
| **NEGmir~ever_good**    | good    |        13,423 |    3.90 |    0.50 |   1.00 | 402.64 |   299 | 291,732 |    300 |
| **NEGmir~ever_black**   | black   |           646 |    1.56 |    0.50 |   1.00 |  77.64 |    56 | 291,732 |     56 |
| **NEGmir~ever_able**    | able    |         1,891 |    2.71 |    0.49 |   0.99 | 178.12 |   136 | 291,732 |    137 |
| **NEGmir~ever_right**   | right   |         2,038 |    1.33 |    0.50 |   1.00 |  67.93 |    49 | 291,732 |     49 |



 ### `2024-05-23` Top 10 "most negative" bigrams corresponding to top 5 adverbs



#### 1. _necessarily_


Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                       | `adj`          |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:--------------------------------------|:---------------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~necessarily_sure**           | sure           |    844,981.00 |    5.91 |    0.95 |  1,436.68 |   222 | 3,226,213 |    224 |
 | **NEGany~necessarily_surprising**     | surprising     |    150,067.00 |    7.22 |    0.93 |  2,150.86 |   343 | 3,226,213 |    355 |
 | **NEGany~necessarily_indicative**     | indicative     |     12,760.00 |    8.37 |    0.93 |  8,811.69 | 1,406 | 3,226,213 |  1,456 |
 | **NEGany~necessarily_representative** | representative |     25,187.00 |    7.31 |    0.91 |  3,044.27 |   496 | 3,226,213 |    524 |
 | **NEGany~necessarily_available**      | available      |    866,272.00 |    6.36 |    0.89 |  1,280.24 |   213 | 3,226,213 |    230 |
 | **NEGany~necessarily_easy**           | easy           |    771,307.00 |    7.26 |    0.88 |  5,448.34 |   914 | 3,226,213 |    996 |
 | **NEGany~necessarily_true**           | true           |    348,994.00 |    6.89 |    0.82 | 18,199.76 | 3,238 | 3,226,213 |  3,786 |
 | **NEGany~necessarily_illegal**        | illegal        |     44,028.00 |    6.48 |    0.87 |  1,659.90 |   280 | 3,226,213 |    307 |
 | **NEGany~necessarily_related**        | related        |    137,661.00 |    6.74 |    0.84 |  4,271.76 |   742 | 3,226,213 |    842 |
 | **NEGany~necessarily_interested**     | interested     |    364,497.00 |    6.77 |    0.87 |  2,500.26 |   422 | 3,226,213 |    463 |


Top 3 `NEGmirror` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                              | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `G2` |   `f` |    `f1` |   `f2` |
 |:-----------------------------|:--------|--------------:|--------:|--------:|-------:|------:|--------:|-------:|
 | **NEGmir~necessarily_wrong** | wrong   |     20,866.00 |    4.27 |    0.81 | 708.98 |   209 | 289,770 |    214 |
 | **NEGmir~necessarily_bad**   | bad     |     10,783.00 |    2.02 |    0.76 | 153.43 |    50 | 289,770 |     54 |
 | **NEGmir~necessarily_true**  | true    |      7,402.00 |    2.18 |    0.75 | 159.07 |    53 | 289,770 |     58 |


#### 2. _exactly_


Top 10 `RBdirect` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                               | `adj`      |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:------------------------------|:-----------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~exactly_surprising** | surprising |    150,067.00 |    7.34 |    0.96 |  2,863.35 |   441 | 3,226,213 |    444 |
 | **NEGany~exactly_cheap**      | cheap      |     83,765.00 |    8.28 |    0.95 |  4,443.27 |   693 | 3,226,213 |    704 |
 | **NEGany~exactly_subtle**     | subtle     |     56,845.00 |    6.92 |    0.94 |  1,671.02 |   264 | 3,226,213 |    271 |
 | **NEGany~exactly_fun**        | fun        |    224,457.00 |    6.67 |    0.94 |  1,423.92 |   225 | 3,226,213 |    231 |
 | **NEGany~exactly_conducive**  | conducive  |     16,405.00 |    6.56 |    0.93 |  1,313.09 |   208 | 3,226,213 |    214 |
 | **NEGany~exactly_sure**       | sure       |    844,981.00 |    8.63 |    0.92 | 54,750.58 | 8,860 | 3,226,213 |  9,301 |
 | **NEGany~exactly_new**        | new        |    321,311.00 |    8.54 |    0.93 |  8,697.93 | 1,378 | 3,226,213 |  1,418 |
 | **NEGany~exactly_easy**       | easy       |    771,307.00 |    8.37 |    0.93 |  6,747.64 | 1,069 | 3,226,213 |  1,100 |
 | **NEGany~exactly_clear**      | clear      |    491,108.00 |    8.30 |    0.92 | 10,937.16 | 1,759 | 3,226,213 |  1,835 |
 | **NEGany~exactly_happy**      | happy      |    528,511.00 |    7.16 |    0.90 |  2,694.69 |   441 | 3,226,213 |    468 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 3. _that_


Top 10 `RBdirect` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                             | `adj`       |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:----------------------------|:------------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~that_uncommon**    | uncommon    |     61,767.00 |    8.39 |    0.94 |  5,136.91 |   804 | 3,226,213 |    819 |
 | **NEGany~that_fond**        | fond        |     39,809.00 |    7.27 |    0.94 |  2,127.94 |   334 | 3,226,213 |    341 |
 | **NEGany~that_surprising**  | surprising  |    150,067.00 |    8.14 |    0.92 |  7,115.30 | 1,141 | 3,226,213 |  1,187 |
 | **NEGany~that_common**      | common      |    556,435.00 |    8.12 |    0.92 |  7,564.08 | 1,216 | 3,226,213 |  1,268 |
 | **NEGany~that_dissimilar**  | dissimilar  |      8,816.00 |    7.00 |    0.92 |  1,904.15 |   307 | 3,226,213 |    321 |
 | **NEGany~that_hard**        | hard        |    430,990.00 |    7.96 |    0.88 | 59,642.82 | 9,966 | 3,226,213 | 10,818 |
 | **NEGany~that_complicated** | complicated |    180,071.00 |    7.95 |    0.91 |  7,450.89 | 1,208 | 3,226,213 |  1,270 |
 | **NEGany~that_impressed**   | impressed   |    113,281.00 |    7.57 |    0.91 |  4,207.58 |   684 | 3,226,213 |    721 |
 | **NEGany~that_noticeable**  | noticeable  |     40,372.00 |    6.78 |    0.91 |  1,632.07 |   265 | 3,226,213 |    279 |
 | **NEGany~that_exciting**    | exciting    |    236,396.00 |    7.48 |    0.90 |  4,892.83 |   805 | 3,226,213 |    859 |


Top 10 `NEGmirror` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                            | `adj`      |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:---------------------------|:-----------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~that_popular**    | popular    |      5,787.00 |    2.50 |    0.76 |   200.44 |    65 | 289,770 |     70 |
 | **NEGmir~that_interested** | interested |      9,258.00 |    2.42 |    0.76 |   190.06 |    62 | 289,770 |     67 |
 | **NEGmir~that_difficult**  | difficult  |     16,043.00 |    2.15 |    0.75 |   155.64 |    52 | 289,770 |     57 |
 | **NEGmir~that_hard**       | hard       |      7,311.00 |    2.31 |    0.74 |   168.31 |    57 | 289,770 |     63 |
 | **NEGmir~that_close**      | close      |     13,962.00 |    2.39 |    0.73 |   174.26 |    60 | 289,770 |     67 |
 | **NEGmir~that_simple**     | simple     |     25,382.00 |    4.34 |    0.73 | 1,370.94 |   473 | 289,770 |    529 |
 | **NEGmir~that_easy**       | easy       |     20,050.00 |    4.21 |    0.72 | 1,258.15 |   442 | 289,770 |    500 |
 | **NEGmir~that_great**      | great      |      5,819.00 |    3.52 |    0.67 |   728.46 |   282 | 289,770 |    340 |
 | **NEGmir~that_good**       | good       |     33,540.00 |    3.07 |    0.56 |   953.31 |   447 | 289,770 |    615 |
 | **NEGmir~that_big**        | big        |      7,859.00 |    3.06 |    0.70 |   309.58 |   113 | 289,770 |    131 |


#### 4. _before_

 No bigrams found in loaded `RBdirect` AM table.
 No bigrams found in loaded `NEGmirror` AM table.

#### 5. _any_


Top 10 `RBdirect` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |      `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|---------:|------:|----------:|-------:|
 | **NEGany~any_happier**   | happier   |     19,501.00 |    4.65 |    0.53 | 3,488.76 |   830 | 3,226,213 |  1,472 |
 | **NEGany~any_simpler**   | simpler   |     26,094.00 |    3.09 |    0.30 |   671.74 |   228 | 3,226,213 |    672 |
 | **NEGany~any_clearer**   | clearer   |     13,369.00 |    3.21 |    0.30 | 1,051.22 |   357 | 3,226,213 |  1,053 |
 | **NEGany~any_different** | different |    909,864.00 |    2.98 |    0.24 | 2,270.24 |   910 | 3,226,213 |  3,313 |
 | **NEGany~any_younger**   | younger   |     29,805.00 |    2.37 |    0.19 |   544.17 |   256 | 3,226,213 |  1,121 |
 | **NEGany~any_worse**     | worse     |    214,166.00 |    2.47 |    0.16 | 3,165.88 | 1,693 | 3,226,213 |  8,487 |
 | **NEGany~any_bigger**    | bigger    |    130,470.00 |    2.27 |    0.17 |   688.06 |   357 | 3,226,213 |  1,735 |
 | **NEGany~any_harder**    | harder    |     99,332.00 |    1.98 |    0.15 |   395.22 |   227 | 3,226,213 |  1,221 |
 | **NEGany~any_safer**     | safer     |     26,779.00 |    1.73 |    0.12 |   346.68 |   235 | 3,226,213 |  1,471 |
 | **NEGany~any_easier**    | easier    |    237,680.00 |    1.95 |    0.11 | 2,164.75 | 1,607 | 3,226,213 | 10,860 |


Top 4 `NEGmirror` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                       | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:----------------------|:--------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~any_better** | better  |     14,076.00 |    4.44 |    0.75 | 1,148.18 |   381 | 289,770 |    416 |
 | **NEGmir~any_easier** | easier  |      2,409.00 |    2.42 |    0.75 |   181.98 |    61 | 289,770 |     67 |
 | **NEGmir~any_worse**  | worse   |      8,490.00 |    2.87 |    0.72 |   248.63 |    88 | 289,770 |    100 |
 | **NEGmir~any_closer** | closer  |        986.00 |    2.21 |    0.68 |   149.62 |    56 | 289,770 |     66 |


#### 6. _ever_


Top 5 `RBdirect` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                         | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |      `f1` |   `f2` |
 |:------------------------|:--------|--------------:|--------:|--------:|---------:|------:|----------:|-------:|
 | **NEGany~ever_simple**  | simple  |    427,167.00 |    5.54 |    0.77 | 1,142.04 |   212 | 3,226,213 |    262 |
 | **NEGany~ever_easy**    | easy    |    771,307.00 |    5.06 |    0.63 | 2,030.58 |   430 | 3,226,213 |    641 |
 | **NEGany~ever_good**    | good    |  2,037,285.00 |    3.76 |    0.40 | 1,178.00 |   332 | 3,226,213 |    756 |
 | **NEGany~ever_perfect** | perfect |    164,519.00 |    3.48 |    0.37 |   736.05 |   217 | 3,226,213 |    527 |
 | **NEGany~ever_able**    | able    |    428,268.00 |    1.81 |    0.13 |   363.95 |   234 | 3,226,213 |  1,398 |


Top 6 `NEGmirror` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                         | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:------------------------|:--------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~ever_easy**    | easy    |     20,050.00 |    3.21 |    0.83 | 1,311.83 |   367 | 289,770 |    368 |
 | **NEGmir~ever_perfect** | perfect |      3,708.00 |    2.38 |    0.83 |   735.10 |   207 | 289,770 |    208 |
 | **NEGmir~ever_good**    | good    |     33,540.00 |    4.72 |    0.82 | 1,034.95 |   298 | 289,770 |    302 |
 | **NEGmir~ever_wrong**   | wrong   |     20,866.00 |    2.56 |    0.82 |   349.21 |   102 | 289,770 |    104 |
 | **NEGmir~ever_free**    | free    |      5,043.00 |    1.97 |    0.81 |   231.61 |    69 | 289,770 |     71 |
 | **NEGmir~ever_able**    | able    |      6,448.00 |    3.66 |    0.79 |   437.65 |   136 | 289,770 |    143 |


#### 7. _yet_


Top 10 `RBdirect` "yet_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |      `G2` |    `f` |      `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|----------:|-------:|----------:|-------:|
 | **NEGany~yet_clear**     | clear     |    491,108.00 |   10.26 |    0.95 | 67,924.56 | 10,553 | 3,226,213 | 10,693 |
 | **NEGany~yet_eligible**  | eligible  |     49,578.00 |    7.72 |    0.94 |  2,929.15 |    459 | 3,226,213 |    468 |
 | **NEGany~yet_official**  | official  |      9,778.00 |    7.33 |    0.94 |  2,236.98 |    353 | 3,226,213 |    362 |
 | **NEGany~yet_ready**     | ready     |    240,297.00 |    9.23 |    0.93 | 48,012.06 |  7,611 | 3,226,213 |  7,838 |
 | **NEGany~yet_certain**   | certain   |    104,544.00 |    8.12 |    0.93 |  5,491.41 |    874 | 3,226,213 |    903 |
 | **NEGany~yet_complete**  | complete  |    107,018.00 |    8.42 |    0.92 | 13,815.99 |  2,220 | 3,226,213 |  2,314 |
 | **NEGany~yet_sure**      | sure      |    844,981.00 |    8.37 |    0.92 | 12,379.79 |  1,990 | 3,226,213 |  2,075 |
 | **NEGany~yet_available** | available |    866,272.00 |    7.69 |    0.87 | 44,196.15 |  7,481 | 3,226,213 |  8,238 |
 | **NEGany~yet_right**     | right     |    204,572.00 |    6.50 |    0.92 |  1,254.20 |    202 | 3,226,213 |    211 |
 | **NEGany~yet_final**     | final     |      9,657.00 |    7.45 |    0.91 |  4,028.75 |    659 | 3,226,213 |    699 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 8. _longer_


Top 5 `RBdirect` "longer_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `G2` |    `f` |       `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|-------:|-------:|-----------:|-------:|
 | **COM~longer_lasting**   | lasting   |     24,344.00 |    1.44 |    0.04 | 244.09 |  3,860 | 83,102,035 |  3,866 |
 | **COM~longer_enough**    | enough    |    453,790.00 |    1.41 |    0.03 | 216.98 |  3,952 | 83,102,035 |  3,964 |
 | **COM~longer_able**      | able      |    428,268.00 |    2.28 |    0.03 | 623.67 | 11,677 | 83,102,035 | 11,716 |
 | **COM~longer_available** | available |    866,272.00 |    2.45 |    0.03 | 974.55 | 18,865 | 83,102,035 | 18,935 |
 | **COM~longer_necessary** | necessary |    187,396.00 |    1.27 |    0.03 | 220.07 |  5,365 | 83,102,035 |  5,399 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 9. _immediately_


Top 5 `RBdirect` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                  | `adj`     |   `adj_total` |   `LRC` |   `dP1` |       `G2` |    `f` |      `f1` |   `f2` |
 |:---------------------------------|:----------|--------------:|--------:|--------:|-----------:|-------:|----------:|-------:|
 | **NEGany~immediately_possible**  | possible  |    364,265.00 |    7.68 |    0.90 |   6,269.26 |  1,027 | 3,226,213 |  1,091 |
 | **NEGany~immediately_clear**     | clear     |    491,108.00 |    8.32 |    0.90 | 153,302.22 | 25,276 | 3,226,213 | 27,066 |
 | **NEGany~immediately_available** | available |    866,272.00 |    5.77 |    0.66 | 102,962.94 | 21,297 | 3,226,213 | 30,725 |
 | **NEGany~immediately_able**      | able      |    428,268.00 |    4.87 |    0.58 |   2,851.84 |    639 | 3,226,213 |  1,036 |
 | **NEGany~immediately_obvious**   | obvious   |    193,498.00 |    4.59 |    0.49 |   9,043.23 |  2,258 | 3,226,213 |  4,305 |


Top 1 `NEGmirror` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                  | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `G2` |   `f` |    `f1` |   `f2` |
 |:---------------------------------|:----------|--------------:|--------:|--------:|-------:|------:|--------:|-------:|
 | **NEGmir~immediately_available** | available |     12,636.00 |    1.94 |    0.43 | 254.47 |   162 | 289,770 |    274 |



In [8]:
bigram_dfs['RBdirect'].filter(like='~before_', axis=0)

,l2,f,E11,am_log_likelihood,...,adj,adv_total,adj_total,l1
key,,,,,,,,,
NEGany~before_available,before_available,177,88.50,245.38,...,available,323,82956,NEGATED


In [9]:
# %%

for key, info in samples_dict.items():
    if key in ('bigrams', 'adj'):
        key = f'ALL {key.replace("adj", "adjectives")}'
    formatted_iter = [
        f'_{a.replace("_", " ")}_' for a
        in (info['adj'] if isinstance(info, dict)
            else info)]
    print_iter(formatted_iter,
               header=f'1. _{key}_ ({len(formatted_iter)} unique)',
               bullet='1.', indent=3)


1. _necessarily_ (11 unique)
   1. _surprising_
   1. _easy_
   1. _enough_
   1. _aware_
   1. _wrong_
   1. _indicative_
   1. _better_
   1. _related_
   1. _true_
   1. _bad_
   1. _new_

1. _that_ (17 unique)
   1. _surprising_
   1. _great_
   1. _big_
   1. _easy_
   1. _exciting_
   1. _unusual_
   1. _uncommon_
   1. _hard_
   1. _difficult_
   1. _popular_
   1. _interested_
   1. _happy_
   1. _simple_
   1. _good_
   1. _different_
   1. _close_
   1. _impressed_

1. _exactly_ (10 unique)
   1. _surprising_
   1. _subtle_
   1. _fun_
   1. _clear_
   1. _easy_
   1. _hard_
   1. _cheap_
   1. _fair_
   1. _sure_
   1. _new_

1. _any_ (10 unique)
   1. _easier_
   1. _sweeter_
   1. _happier_
   1. _brighter_
   1. _better_
   1. _nicer_
   1. _different_
   1. _smarter_
   1. _younger_
   1. _worse_

1. _remotely_ (10 unique)
   1. _surprising_
   1. _comparable_
   1. _similar_
   1. _enough_
   1. _interested_
   1. _ready_
   1. _funny_
   1. _close_
   1. _true_
   1. 


1. _necessarily_ (11 unique)
   1. _surprising_
   1. _easy_
   1. _enough_
   1. _aware_
   1. _wrong_
   1. _indicative_
   1. _better_
   1. _related_
   1. _true_
   1. _bad_
   1. _new_

1. _that_ (17 unique)
   1. _surprising_
   1. _great_
   1. _big_
   1. _easy_
   1. _exciting_
   1. _unusual_
   1. _uncommon_
   1. _hard_
   1. _difficult_
   1. _popular_
   1. _interested_
   1. _happy_
   1. _simple_
   1. _good_
   1. _different_
   1. _close_
   1. _impressed_

1. _exactly_ (10 unique)
   1. _surprising_
   1. _subtle_
   1. _fun_
   1. _clear_
   1. _easy_
   1. _hard_
   1. _cheap_
   1. _fair_
   1. _sure_
   1. _new_

1. _any_ (10 unique)
   1. _easier_
   1. _sweeter_
   1. _happier_
   1. _brighter_
   1. _better_
   1. _nicer_
   1. _different_
   1. _smarter_
   1. _younger_
   1. _worse_

1. _remotely_ (10 unique)
   1. _surprising_
   1. _comparable_
   1. _similar_
   1. _enough_
   1. _interested_
   1. _ready_
   1. _funny_
   1. _close_
   1. _true_
   1. _related_

1. _yet_ (10 unique)
   1. _dead_
   1. _clear_
   1. _able_
   1. _final_
   1. _complete_
   1. _ready_
   1. _available_
   1. _sure_
   1. _certain_
   1. _public_

1. _immediately_ (10 unique)
   1. _clear_
   1. _possible_
   1. _able_
   1. _obvious_
   1. _available_
   1. _sure_
   1. _certain_
   1. _apparent_
   1. _successful_
   1. _reachable_

1. _particularly_ (15 unique)
   1. _surprising_
   1. _original_
   1. _innovative_
   1. _unusual_
   1. _radical_
   1. _athletic_
   1. _religious_
   1. _novel_
   1. _wrong_
   1. _flashy_
   1. _good_
   1. _comfortable_
   1. _likeable_
   1. _acute_
   1. _new_

1. _inherently_ (6 unique)
   1. _wrong_
   1. _better_
   1. _good_
   1. _illegal_
   1. _evil_
   1. _bad_

1. _terribly_ (14 unique)
   1. _surprising_
   1. _original_
   1. _interesting_
   1. _unusual_
   1. _popular_
   1. _exciting_
   1. _interested_
   1. _common_
   1. _bright_
   1. _wrong_
   1. _surprised_
   1. _different_
   1. _comfortable_
   1. _new_

1. _ever_ (17 unique)
   1. _greater_
   1. _closer_
   1. _easy_
   1. _higher_
   1. _enough_
   1. _black_
   1. _wrong_
   1. _able_
   1. _mindful_
   1. _perfect_
   1. _simple_
   1. _good_
   1. _deeper_
   1. _certain_
   1. _right_
   1. _boring_
   1. _larger_

1. _ALL bigrams_ (130 unique)
   1. _immediately clear_
   1. _ever good_
   1. _yet clear_
   1. _inherently wrong_
   1. _any brighter_
   1. _necessarily indicative_
   1. _that big_
   1. _necessarily easy_
   1. _immediately able_
   1. _that good_
   1. _exactly sure_
   1. _particularly good_
   1. _terribly popular_
   1. _ever deeper_
   1. _any younger_
   1. _particularly acute_
   1. _necessarily aware_
   1. _ever able_
   1. _remotely enough_
   1. _immediately sure_
   1. _particularly flashy_
   1. _yet certain_
   1. _necessarily new_
   1. _yet final_
   1. _that interested_
   1. _terribly different_
   1. _particularly comfortable_
   1. _exactly clear_
   1. _exactly hard_
   1. _ever greater_
   1. _remotely surprising_
   1. _any different_
   1. _exactly new_
   1. _immediately successful_
   1. _necessarily enough_
   1. _remotely funny_
   1. _yet complete_
   1. _ever wrong_
   1. _that unusual_
   1. _that hard_
   1. _yet sure_
   1. _yet able_
   1. _particularly religious_
   1. _remotely true_
   1. _necessarily better_
   1. _exactly subtle_
   1. _ever easy_
   1. _that easy_
   1. _ever right_
   1. _remotely ready_
   1. _immediately possible_
   1. _terribly original_
   1. _that great_
   1. _terribly exciting_
   1. _that popular_
   1. _terribly surprised_
   1. _ever perfect_
   1. _terribly interesting_
   1. _remotely related_
   1. _inherently good_
   1. _immediately reachable_
   1. _immediately certain_
   1. _that exciting_
   1. _exactly fair_
   1. _that happy_
   1. _any smarter_
   1. _particularly wrong_
   1. _terribly surprising_
   1. _any easier_
   1. _terribly new_
   1. _immediately obvious_
   1. _particularly radical_
   1. _that impressed_
   1. _inherently illegal_
   1. _any nicer_
   1. _terribly unusual_
   1. _any happier_
   1. _ever larger_
   1. _terribly wrong_
   1. _any sweeter_
   1. _necessarily surprising_
   1. _remotely comparable_
   1. _ever mindful_
   1. _necessarily wrong_
   1. _exactly fun_
   1. _particularly athletic_
   1. _ever closer_
   1. _terribly comfortable_
   1. _particularly innovative_
   1. _that uncommon_
   1. _particularly novel_
   1. _particularly surprising_
   1. _immediately available_
   1. _particularly new_
   1. _ever higher_
   1. _ever black_
   1. _inherently bad_
   1. _necessarily bad_
   1. _particularly unusual_
   1. _that simple_
   1. _any worse_
   1. _ever certain_
   1. _inherently evil_
   1. _that surprising_
   1. _necessarily true_
   1. _immediately apparent_
   1. _ever boring_
   1. _ever simple_
   1. _terribly interested_
   1. _necessarily related_
   1. _remotely close_
   1. _that different_
   1. _any better_
   1. _particularly likeable_
   1. _that close_
   1. _exactly cheap_
   1. _remotely similar_
   1. _yet dead_
   1. _yet ready_
   1. _particularly original_
   1. _that difficult_
   1. _ever enough_
   1. _inherently better_
   1. _remotely interested_
   1. _yet public_
   1. _terribly bright_
   1. _terribly common_
   1. _yet available_
   1. _exactly surprising_
   1. _exactly easy_

1. _ALL adjectives_ (82 unique)
   1. _great_
   1. _higher_
   1. _brighter_
   1. _cheap_
   1. _comparable_
   1. _interested_
   1. _bright_
   1. _available_
   1. _related_
   1. _larger_
   1. _enough_
   1. _black_
   1. _perfect_
   1. _reachable_
   1. _greater_
   1. _final_
   1. _simple_
   1. _fair_
   1. _comfortable_
   1. _worse_
   1. _big_
   1. _clear_
   1. _easy_
   1. _exciting_
   1. _able_
   1. _better_
   1. _nicer_
   1. _apparent_
   1. _true_
   1. _indicative_
   1. _closer_
   1. _unusual_
   1. _popular_
   1. _novel_
   1. _obvious_
   1. _uncommon_
   1. _sweeter_
   1. _mindful_
   1. _deeper_
   1. _impressed_
   1. _younger_
   1. _acute_
   1. _happier_
   1. _complete_
   1. _surprised_
   1. _certain_
   1. _smarter_
   1. _easier_
   1. _possible_
   1. _happy_
   1. _boring_
   1. _surprising_
   1. _close_
   1. _new_
   1. _public_
   1. _subtle_
   1. _similar_
   1. _common_
   1. _athletic_
   1. _wrong_
   1. _original_
   1. _bad_
   1. _dead_
   1. _hard_
   1. _ready_
   1. _sure_
   1. _different_
   1. _successful_
   1. _evil_
   1. _fun_
   1. _religious_
   1. _flashy_
   1. _likeable_
   1. _illegal_
   1. _difficult_
   1. _innovative_
   1. _interesting_
   1. _radical_
   1. _aware_
   1. _right_
   1. _good_
   1. _funny_


In [10]:
NEG_bigrams_sample = pd.concat(
    (ad['both'] for ad in samples_dict.values() if isinstance(ad, dict))).sort_values('LRC', ascending=False)

In [11]:
top_NEGbigram_df_path =  OUT_DIR.joinpath(
    f'{TAG}-Top{K}_NEG-ADV_top-{bigram_k}-bigrams-{bigram_floor}.{timestamp_today()}.csv')
print(top_NEGbigram_df_path)


/share/compling/projects/sanpi/results/top_AM/NEQ/NEQ-Top8/NEQ-Top8_NEG-ADV_top-10-bigrams-25.2024-07-28.csv


In [12]:
NEG_bigrams_sample.to_csv(top_NEGbigram_df_path)
nb_show_table(NEG_bigrams_sample.sort_values('LRC', ascending=False),
              outpath=top_NEGbigram_df_path.with_suffix('.md'))


|                                     |   `LRC` |   `MI` |   `dP1` |   `P1` |    `f` |   `adv_total` |   `f2` | `adv`        |   `unexp_f` |   `odds_r_disc` | `l1`       |      `f1` |   `t` |   `adj_total` |       `N` | `adj`       |   `exp_f` | `l2`                     |      `G2` |
|:------------------------------------|--------:|-------:|--------:|-------:|-------:|--------------:|-------:|:-------------|------------:|----------------:|:-----------|----------:|------:|--------------:|----------:|:------------|----------:|:-------------------------|----------:|
| **NEGany~yet_clear**                |    8.66 |   0.30 |    0.50 |   1.00 | 10,406 |        53,881 | 10,409 | yet          |    5,201.54 |            3.47 | NEGATED    | 3,173,660 | 50.99 |        84,227 | 6,347,364 | clear       |  5,204.46 | yet_clear                | 14,392.25 |
| **NEGany~yet_ready**                |    8.06 |   0.30 |    0.50 |   1.00 |  7,501 |        53,881 |  7,505 | yet          |    3,748.53 |    


|                                     |   `LRC` |   `MI` |   `dP1` |   `P1` |    `f` |   `adv_total` |   `f2` | `adv`        |   `unexp_f` |   `odds_r_disc` | `l1`       |      `f1` |   `t` |   `adj_total` |       `N` | `adj`       |   `exp_f` | `l2`                     |      `G2` |
|:------------------------------------|--------:|-------:|--------:|-------:|-------:|--------------:|-------:|:-------------|------------:|----------------:|:-----------|----------:|------:|--------------:|----------:|:------------|----------:|:-------------------------|----------:|
| **NEGany~yet_clear**                |    8.66 |   0.30 |    0.50 |   1.00 | 10,406 |        53,881 | 10,409 | yet          |    5,201.54 |            3.47 | NEGATED    | 3,173,660 | 50.99 |        84,227 | 6,347,364 | clear       |  5,204.46 | yet_clear                | 14,392.25 |
| **NEGany~yet_ready**                |    8.06 |   0.30 |    0.50 |   1.00 |  7,501 |        53,881 |  7,505 | yet          |    3,748.53 |            3.22 | NEGATED    | 3,173,660 | 43.28 |        29,583 | 6,347,364 | ready       |  3,752.47 | yet_ready                | 10,344.81 |
| **NEGany~that_hard**                |    7.68 |   0.30 |    0.50 |   1.00 |  9,948 |       166,676 |  9,963 | that         |    4,966.53 |            2.81 | NEGATED    | 3,173,660 | 49.79 |        45,061 | 6,347,364 | hard        |  4,981.47 | that_hard                | 13,602.42 |
| **NEGany~immediately_clear**        |    7.55 |   0.30 |    0.50 |   1.00 | 24,416 |        58,040 | 24,488 | immediately  |   12,172.08 |            2.53 | NEGATED    | 3,173,660 | 77.90 |        84,227 | 6,347,364 | clear       | 12,243.92 | immediately_clear        | 33,058.44 |
| **NEGany~exactly_sure**             |    7.46 |   0.30 |    0.50 |   1.00 |  8,794 |        44,503 |  8,810 | exactly      |    4,389.03 |            2.73 | NEGATED    | 3,173,660 | 46.80 |       134,139 | 6,347,364 | sure        |  4,404.97 | exactly_sure             | 11,991.61 |
| **NEGany~that_different**           |    7.18 |   0.30 |    0.50 |   1.00 |  6,534 |       166,676 |  6,547 | that         |    3,260.52 |            2.69 | NEGATED    | 3,173,660 | 40.34 |        80,643 | 6,347,364 | different   |  3,273.48 | that_different           |  8,895.12 |
| **NEGany~that_great**               |    7.18 |   0.30 |    0.50 |   1.00 | 11,032 |       166,676 | 11,065 | that         |    5,499.54 |            2.52 | NEGATED    | 3,173,660 | 52.36 |        45,359 | 6,347,364 | great       |  5,532.46 | that_great               | 14,908.90 |
| **NEGany~that_difficult**           |    7.06 |   0.30 |    0.50 |   1.00 |  5,560 |       166,676 |  5,571 | that         |    2,774.52 |            2.69 | NEGATED    | 3,173,660 | 37.21 |        61,490 | 6,347,364 | difficult   |  2,785.48 | that_difficult           |  7,569.00 |
| **NEGany~yet_complete**             |    6.70 |   0.30 |    0.50 |   1.00 |  2,174 |        53,881 |  2,175 | yet          |    1,086.51 |            3.16 | NEGATED    | 3,173,660 | 23.30 |         8,415 | 6,347,364 | complete    |  1,087.49 | yet_complete             |  2,998.60 |
| **NEGany~yet_available**            |    6.66 |   0.30 |    0.50 |   1.00 |  7,430 |        53,881 |  7,461 | yet          |    3,699.53 |            2.37 | NEGATED    | 3,173,660 | 42.92 |        82,956 | 6,347,364 | available   |  3,730.47 | yet_available            |  9,950.03 |
| **NEGany~that_big**                 |    6.47 |   0.30 |    0.50 |   1.00 |  6,244 |       166,676 |  6,273 | that         |    3,107.52 |            2.33 | NEGATED    | 3,173,660 | 39.33 |        42,912 | 6,347,364 | big         |  3,136.48 | that_big                 |  8,332.69 |
| **NEGany~exactly_clear**            |    6.38 |   0.30 |    0.50 |   1.00 |  1,746 |        44,503 |  1,747 | exactly      |      872.51 |            3.07 | NEGATED    | 3,173,660 | 20.88 |        84,227 | 6,347,364 | clear       |    873.49 | exactly_clear            |  2,405.43 |
| **NEGany~necessarily_bad**          |    6.31 |   0.30 |    0.50 |   1.00 |  2,059 |        42,886 |  2,062 | necessarily  |    1,028.01 |            2.77 | NEGATED    | 3,173,660 | 22.66 |       119,509 | 6,347,364 | bad         |  1,030.99 | necessarily_bad          |  2,814.04 |
| **NEGany~necessarily_indicative**   |    6.29 |   0.30 |    0.50 |   1.00 |  1,389 |        42,886 |  1,389 | necessarily  |      694.50 |            3.44 | NEGATED    | 3,173,660 | 18.63 |         2,313 | 6,347,364 | indicative  |    694.50 | necessarily_indicative   |  1,925.89 |
| **NEGany~necessarily_true**         |    6.16 |   0.30 |    0.50 |   1.00 |  3,232 |        42,886 |  3,245 | necessarily  |    1,609.51 |            2.38 | NEGATED    | 3,173,660 | 28.31 |        34,967 | 6,347,364 | true        |  1,622.49 | necessarily_true         |  4,330.74 |
| **NEGany~yet_sure**                 |    6.13 |   0.30 |    0.50 |   1.00 |  1,977 |        53,881 |  1,981 | yet          |      986.51 |            2.64 | NEGATED    | 3,173,660 | 22.19 |       134,139 | 6,347,364 | sure        |    990.49 | yet_sure                 |  2,689.26 |
| **NEGany~necessarily_better**       |    6.07 |   0.30 |    0.50 |   1.00 |  1,887 |        42,886 |  1,891 | necessarily  |      941.51 |            2.62 | NEGATED    | 3,173,660 | 21.67 |        50,827 | 6,347,364 | better      |    945.49 | necessarily_better       |  2,564.81 |
| **NEGany~exactly_new**              |    6.03 |   0.30 |    0.50 |   1.00 |  1,371 |        44,503 |  1,372 | exactly      |      685.00 |            2.96 | NEGATED    | 3,173,660 | 18.50 |        21,538 | 6,347,364 | new         |    686.00 | exactly_new              |  1,885.86 |
| **NEGany~that_surprising**          |    5.99 |   0.30 |    0.50 |   1.00 |  1,133 |       166,676 |  1,133 | that         |      566.50 |            3.36 | NEGATED    | 3,173,660 | 16.83 |        18,776 | 6,347,364 | surprising  |    566.50 | that_surprising          |  1,570.89 |
| **NEGany~that_unusual**             |    5.77 |   0.30 |    0.50 |   1.00 |    977 |       166,676 |    977 | that         |      488.50 |            3.29 | NEGATED    | 3,173,660 | 15.63 |         7,412 | 6,347,364 | unusual     |    488.50 | that_unusual             |  1,354.57 |
| **NEGany~terribly_surprising**      |    5.73 |   0.30 |    0.50 |   1.00 |    949 |        19,802 |    949 | terribly     |      474.50 |            3.28 | NEGATED    | 3,173,660 | 15.40 |        18,776 | 6,347,364 | surprising  |    474.50 | terribly_surprising      |  1,315.75 |
| **NEGany~necessarily_easy**         |    5.67 |   0.30 |    0.50 |   1.00 |    909 |        42,886 |    909 | necessarily  |      454.50 |            3.26 | NEGATED    | 3,173,660 | 15.07 |       108,923 | 6,347,364 | easy        |    454.50 | necessarily_easy         |  1,260.28 |
| **NEGany~exactly_easy**             |    5.67 |   0.30 |    0.50 |   1.00 |  1,066 |        44,503 |  1,067 | exactly      |      532.50 |            2.85 | NEGATED    | 3,173,660 | 16.31 |       108,923 | 6,347,364 | easy        |    533.50 | exactly_easy             |  1,463.43 |
| **NEGany~yet_certain**              |    5.60 |   0.30 |    0.50 |   1.00 |    866 |        53,881 |    866 | yet          |      433.00 |            3.24 | NEGATED    | 3,173,660 | 14.71 |        11,334 | 6,347,364 | certain     |    433.00 | yet_certain              |  1,200.66 |
| **NEGany~that_exciting**            |    5.49 |   0.30 |    0.50 |   1.00 |    805 |       166,676 |    805 | that         |      402.50 |            3.21 | NEGATED    | 3,173,660 | 14.19 |        20,233 | 6,347,364 | exciting    |    402.50 | that_exciting            |  1,116.08 |
| **NEGany~that_uncommon**            |    5.49 |   0.30 |    0.50 |   1.00 |    802 |       166,676 |    802 | that         |      401.00 |            3.21 | NEGATED    | 3,173,660 | 14.16 |         3,165 | 6,347,364 | uncommon    |    401.00 | that_uncommon            |  1,111.92 |
| **NEGany~yet_able**                 |    5.44 |   0.30 |    0.50 |   1.00 |  1,315 |        53,881 |  1,320 | yet          |      655.00 |            2.38 | NEGATED    | 3,173,660 | 18.06 |        23,355 | 6,347,364 | able        |    660.00 | yet_able                 |  1,764.46 |
| **NEGany~immediately_possible**     |    5.40 |   0.30 |    0.50 |   1.00 |  1,000 |        58,040 |  1,002 | immediately  |      499.00 |            2.60 | NEGATED    | 3,173,660 | 15.78 |        30,446 | 6,347,364 | possible    |    501.00 | immediately_possible     |  1,360.38 |
| **NEGany~immediately_available**    |    5.34 |   0.29 |    0.48 |   0.98 | 21,078 |        58,040 | 21,477 | immediately  |   10,339.57 |            1.73 | NEGATED    | 3,173,660 | 71.22 |        82,956 | 6,347,364 | available   | 10,738.43 | immediately_available    | 25,870.14 |
| **NEGany~exactly_cheap**            |    5.27 |   0.30 |    0.50 |   1.00 |    691 |        44,503 |    691 | exactly      |      345.50 |            3.14 | NEGATED    | 3,173,660 | 13.14 |         6,591 | 6,347,364 | cheap       |    345.50 | exactly_cheap            |    958.01 |
| **NEGany~that_impressed**           |    5.25 |   0.30 |    0.50 |   1.00 |    681 |       166,676 |    681 | that         |      340.50 |            3.13 | NEGATED    | 3,173,660 | 13.05 |        12,138 | 6,347,364 | impressed   |    340.50 | that_impressed           |    944.15 |
| **NEGany~yet_final**                |    5.16 |   0.30 |    0.50 |   1.00 |    640 |        53,881 |    640 | yet          |      320.00 |            3.11 | NEGATED    | 3,173,660 | 12.65 |         1,213 | 6,347,364 | final       |    320.00 | yet_final                |    887.30 |
| **NEGany~necessarily_related**      |    5.14 |   0.30 |    0.50 |   1.00 |    741 |        42,886 |    742 | necessarily  |      370.00 |            2.69 | NEGATED    | 3,173,660 | 13.59 |        14,260 | 6,347,364 | related     |    371.00 | necessarily_related      |  1,013.51 |
| **NEGany~necessarily_new**          |    4.74 |   0.30 |    0.50 |   1.00 |    482 |        42,886 |    482 | necessarily  |      241.00 |            2.98 | NEGATED    | 3,173,660 | 10.98 |        21,538 | 6,347,364 | new         |    241.00 | necessarily_new          |    668.24 |
| **NEGany~yet_public**               |    4.69 |   0.30 |    0.50 |   1.00 |    467 |        53,881 |    467 | yet          |      233.50 |            2.97 | NEGATED    | 3,173,660 | 10.81 |         2,656 | 6,347,364 | public      |    233.50 | yet_public               |    647.44 |
| **NEGany~any_happier**              |    4.66 |   0.30 |    0.49 |   0.99 |    828 |        16,238 |    834 | any          |      411.00 |            2.11 | NEGATED    | 3,173,660 | 14.28 |         2,004 | 6,347,364 | happier     |    417.00 | any_happier              |  1,085.12 |
| **NEGany~particularly_new**         |    4.61 |   0.30 |    0.49 |   0.99 |    747 |        76,162 |    752 | particularly |      371.00 |            2.13 | NEGATED    | 3,173,660 | 13.57 |        21,538 | 6,347,364 | new         |    376.00 | particularly_new         |    982.49 |
| **NEGany~exactly_surprising**       |    4.61 |   0.30 |    0.50 |   1.00 |    440 |        44,503 |    440 | exactly      |      220.00 |            2.95 | NEGATED    | 3,173,660 | 10.49 |        18,776 | 6,347,364 | surprising  |    220.00 | exactly_surprising       |    610.01 |
| **NEGany~particularly_religious**   |    4.52 |   0.30 |    0.50 |   1.00 |    485 |        76,162 |    486 | particularly |      242.00 |            2.51 | NEGATED    | 3,173,660 | 10.99 |         3,507 | 6,347,364 | religious   |    243.00 | particularly_religious   |    659.41 |
| **NEGany~yet_dead**                 |    4.47 |   0.30 |    0.50 |   1.00 |    401 |        53,881 |    401 | yet          |      200.50 |            2.90 | NEGATED    | 3,173,660 | 10.01 |         6,348 | 6,347,364 | dead        |    200.50 | yet_dead                 |    555.93 |
| **NEGany~any_easier**               |    4.42 |   0.29 |    0.48 |   0.98 |  1,594 |        16,238 |  1,625 | any          |      781.51 |            1.70 | NEGATED    | 3,173,660 | 19.57 |        12,877 | 6,347,364 | easier      |    812.49 | any_easier               |  1,946.26 |
| **NEGmir~particularly_new**         |    4.35 |   0.30 |    0.50 |   1.00 |    404 |        10,029 |    405 | particularly |      201.50 |            2.43 | NEGMIR     |   291,732 | 10.03 |         4,300 |   583,470 | new         |    202.50 | particularly_new         |    547.73 |
| **NEGany~inherently_wrong**         |    4.25 |   0.29 |    0.48 |   0.98 |  1,639 |         8,614 |  1,678 | inherently   |      800.01 |            1.62 | NEGATED    | 3,173,660 | 19.76 |        21,332 | 6,347,364 | wrong       |    838.99 | inherently_wrong         |  1,956.12 |
| **NEGany~necessarily_surprising**   |    4.23 |   0.30 |    0.50 |   1.00 |    340 |        42,886 |    340 | necessarily  |      170.00 |            2.83 | NEGATED    | 3,173,660 |  9.22 |        18,776 | 6,347,364 | surprising  |    170.00 | necessarily_surprising   |    471.36 |
| **NEGmir~ever_easy**                |    4.21 |   0.30 |    0.50 |   1.00 |    368 |         4,786 |    369 | ever         |      183.50 |            2.39 | NEGMIR     |   291,732 |  9.57 |         7,749 |   583,470 | easy        |    184.50 | ever_easy                |    497.96 |
| **NEGany~terribly_interested**      |    3.98 |   0.30 |    0.49 |   0.99 |    486 |        19,802 |    491 | terribly     |      240.50 |            1.95 | NEGATED    | 3,173,660 | 10.91 |        34,543 | 6,347,364 | interested  |    245.50 | terribly_interested      |    624.89 |
| **NEGany~necessarily_enough**       |    3.93 |   0.30 |    0.50 |   1.00 |    279 |        42,886 |    279 | necessarily  |      139.50 |            2.75 | NEGATED    | 3,173,660 |  8.35 |        27,603 | 6,347,364 | enough      |    139.50 | necessarily_enough       |    386.79 |
| **NEGany~particularly_surprising**  |    3.93 |   0.29 |    0.47 |   0.97 |  1,069 |        76,162 |  1,097 | particularly |      520.50 |            1.57 | NEGATED    | 3,173,660 | 15.92 |        18,776 | 6,347,364 | surprising  |    548.50 | particularly_surprising  |  1,260.26 |
| **NEGany~terribly_different**       |    3.93 |   0.30 |    0.49 |   0.99 |    366 |        19,802 |    368 | terribly     |      182.00 |            2.17 | NEGATED    | 3,173,660 |  9.51 |        80,643 | 6,347,364 | different   |    184.00 | terribly_different       |    485.33 |
| **NEGmir~ever_good**                |    3.90 |   0.30 |    0.50 |   1.00 |    299 |         4,786 |    300 | ever         |      149.00 |            2.30 | NEGMIR     |   291,732 |  8.62 |        13,423 |   583,470 | good        |    150.00 | ever_good                |    402.64 |
| **NEGany~immediately_obvious**      |    3.88 |   0.28 |    0.46 |   0.96 |  2,238 |        58,040 |  2,325 | immediately  |    1,075.51 |            1.41 | NEGATED    | 3,173,660 | 22.73 |        22,651 | 6,347,364 | obvious     |  1,162.49 | immediately_obvious      |  2,481.50 |
| **NEGany~inherently_bad**           |    3.87 |   0.29 |    0.48 |   0.98 |    794 |         8,614 |    812 | inherently   |      388.00 |            1.63 | NEGATED    | 3,173,660 | 13.77 |       119,509 | 6,347,364 | bad         |    406.00 | inherently_bad           |    953.05 |
| **NEGany~exactly_subtle**           |    3.84 |   0.30 |    0.50 |   1.00 |    263 |        44,503 |    263 | exactly      |      131.50 |            2.72 | NEGATED    | 3,173,660 |  8.11 |         5,299 | 6,347,364 | subtle      |    131.50 | exactly_subtle           |    364.61 |
| **NEGany~exactly_fair**             |    3.83 |   0.30 |    0.50 |   1.00 |    260 |        44,503 |    260 | exactly      |      130.00 |            2.72 | NEGATED    | 3,173,660 |  8.06 |         6,964 | 6,347,364 | fair        |    130.00 | exactly_fair             |    360.45 |
| **NEGany~any_younger**              |    3.80 |   0.30 |    0.50 |   1.00 |    255 |        16,238 |    255 | any          |      127.50 |            2.71 | NEGATED    | 3,173,660 |  7.98 |         1,784 | 6,347,364 | younger     |    127.50 | any_younger              |    353.52 |
| **NEGmir~inherently_wrong**         |    3.78 |   0.28 |    0.46 |   0.96 |  1,513 |         3,342 |  1,571 | inherently   |      727.51 |            1.42 | NEGMIR     |   291,732 | 18.70 |         8,506 |   583,470 | wrong       |    785.49 | inherently_wrong         |  1,685.02 |
| **NEGmir~that_simple**              |    3.67 |   0.29 |    0.48 |   0.98 |    474 |         4,559 |    483 | that         |      232.50 |            1.70 | NEGMIR     |   291,732 | 10.68 |         7,465 |   583,470 | simple      |    241.50 | that_simple              |    580.44 |
| **NEGany~immediately_able**         |    3.66 |   0.29 |    0.48 |   0.98 |    626 |        58,040 |    641 | immediately  |      305.50 |            1.61 | NEGATED    | 3,173,660 | 12.21 |        23,355 | 6,347,364 | able        |    320.50 | immediately_able         |    746.39 |
| **NEGany~particularly_original**    |    3.64 |   0.30 |    0.49 |   0.99 |    360 |        76,162 |    364 | particularly |      178.00 |            1.90 | NEGATED    | 3,173,660 |  9.38 |         4,693 | 6,347,364 | original    |    182.00 | particularly_original    |    460.59 |
| **NEGany~any_worse**                |    3.62 |   0.28 |    0.46 |   0.96 |  1,686 |        16,238 |  1,762 | any          |      805.01 |            1.34 | NEGATED    | 3,173,660 | 19.61 |        12,116 | 6,347,364 | worse       |    880.99 | any_worse                |  1,816.60 |
| **NEGany~exactly_fun**              |    3.60 |   0.30 |    0.50 |   1.00 |    224 |        44,503 |    224 | exactly      |      112.00 |            2.65 | NEGATED    | 3,173,660 |  7.48 |        19,661 | 6,347,364 | fun         |    112.00 | exactly_fun              |    310.54 |
| **NEGmir~ever_perfect**             |    3.60 |   0.30 |    0.50 |   1.00 |    206 |         4,786 |    206 | ever         |      103.00 |            2.62 | NEGMIR     |   291,732 |  7.18 |         1,303 |   583,470 | perfect     |    103.00 | ever_perfect             |    285.65 |
| **NEGmir~ever_simple**              |    3.60 |   0.30 |    0.50 |   1.00 |    206 |         4,786 |    206 | ever         |      103.00 |            2.62 | NEGMIR     |   291,732 |  7.18 |         7,465 |   583,470 | simple      |    103.00 | ever_simple              |    285.65 |
| **NEGany~any_better**               |    3.59 |   0.28 |    0.44 |   0.94 |  4,719 |        16,238 |  5,004 | any          |    2,217.02 |            1.22 | NEGATED    | 3,173,660 | 32.27 |        50,827 | 6,347,364 | better      |  2,501.98 | any_better               |  4,753.39 |
| **NEGany~particularly_wrong**       |    3.56 |   0.30 |    0.50 |   1.00 |    218 |        76,162 |    218 | particularly |      109.00 |            2.64 | NEGATED    | 3,173,660 |  7.38 |        21,332 | 6,347,364 | wrong       |    109.00 | particularly_wrong       |    302.22 |
| **NEGany~remotely_true**            |    3.53 |   0.30 |    0.50 |   1.00 |    250 |         6,161 |    251 | remotely     |      124.50 |            2.22 | NEGATED    | 3,173,660 |  7.87 |        34,967 | 6,347,364 | true        |    125.50 | remotely_true            |    334.93 |
| **NEGany~ever_easy**                |    3.53 |   0.29 |    0.48 |   0.98 |    429 |        10,870 |    437 | ever         |      210.50 |            1.70 | NEGATED    | 3,173,660 | 10.16 |       108,923 | 6,347,364 | easy        |    218.50 | ever_easy                |    525.98 |
| **COM~ever_closer**                 |    3.52 |   0.30 |    0.49 |   0.99 |    279 |        10,870 |    281 | ever         |      138.51 |            2.05 | COMPLEMENT | 3,173,552 |  8.29 |         3,686 | 6,347,364 | closer      |    140.49 | ever_closer              |    365.82 |
| **NEGany~necessarily_aware**        |    3.48 |   0.30 |    0.50 |   1.00 |    206 |        42,886 |    206 | necessarily  |      103.00 |            2.62 | NEGATED    | 3,173,660 |  7.18 |        28,973 | 6,347,364 | aware       |    103.00 | necessarily_aware        |    285.59 |
| **NEGany~exactly_hard**             |    3.46 |   0.30 |    0.50 |   1.00 |    203 |        44,503 |    203 | exactly      |      101.50 |            2.61 | NEGATED    | 3,173,660 |  7.12 |        45,061 | 6,347,364 | hard        |    101.50 | exactly_hard             |    281.43 |
| **NEGmir~particularly_wrong**       |    3.39 |   0.30 |    0.50 |   1.00 |    212 |        10,029 |    213 | particularly |      105.50 |            2.15 | NEGMIR     |   291,732 |  7.25 |         8,506 |   583,470 | wrong       |    106.50 | particularly_wrong       |    282.64 |
| **NEGany~immediately_apparent**     |    3.30 |   0.27 |    0.44 |   0.94 |  2,015 |        58,040 |  2,143 | immediately  |      943.51 |            1.20 | NEGATED    | 3,173,660 | 21.02 |         9,798 | 6,347,364 | apparent    |  1,071.49 | immediately_apparent     |  2,001.83 |
| **NEGany~terribly_surprised**       |    3.30 |   0.30 |    0.49 |   0.99 |    287 |        19,802 |    291 | terribly     |      141.50 |            1.81 | NEGATED    | 3,173,660 |  8.35 |        10,157 | 6,347,364 | surprised   |    145.50 | terribly_surprised       |    361.19 |
| **NEGany~terribly_exciting**        |    3.28 |   0.29 |    0.48 |   0.98 |    382 |        19,802 |    391 | terribly     |      186.50 |            1.60 | NEGATED    | 3,173,660 |  9.54 |        20,233 | 6,347,364 | exciting    |    195.50 | terribly_exciting        |    456.39 |
| **NEGany~ever_simple**              |    3.28 |   0.30 |    0.50 |   1.00 |    211 |        10,870 |    212 | ever         |      105.00 |            2.15 | NEGATED    | 3,173,660 |  7.23 |        46,867 | 6,347,364 | simple      |    106.00 | ever_simple              |    281.20 |
| **NEGmir~any_better**               |    3.27 |   0.29 |    0.47 |   0.97 |    380 |         1,095 |    390 | any          |      185.00 |            1.56 | NEGMIR     |   291,732 |  9.49 |         3,831 |   583,470 | better      |    195.00 | any_better               |    447.88 |
| **NEGmir~particularly_surprising**  |    3.27 |   0.30 |    0.50 |   1.00 |    166 |        10,029 |    166 | particularly |       83.00 |            2.52 | NEGMIR     |   291,732 |  6.44 |         1,248 |   583,470 | surprising  |     83.00 | particularly_surprising  |    230.18 |
| **NEGmir~particularly_good**        |    3.24 |   0.29 |    0.47 |   0.97 |    390 |        10,029 |    401 | particularly |      189.50 |            1.53 | NEGMIR     |   291,732 |  9.60 |        13,423 |   583,470 | good        |    200.50 | particularly_good        |    455.35 |
| **NEGmir~that_easy**                |    3.23 |   0.29 |    0.47 |   0.97 |    450 |         4,559 |    465 | that         |      217.50 |            1.46 | NEGMIR     |   291,732 | 10.25 |         7,749 |   583,470 | easy        |    232.50 | that_easy                |    512.43 |
| **NEGmir~exactly_sure**             |    3.10 |   0.30 |    0.50 |   1.00 |    148 |           869 |    148 | exactly      |       74.00 |            2.47 | NEGMIR     |   291,732 |  6.08 |         5,978 |   583,470 | sure        |     74.00 | exactly_sure             |    205.21 |
| **NEGmir~ever_enough**              |    3.09 |   0.30 |    0.50 |   1.00 |    147 |         4,786 |    147 | ever         |       73.50 |            2.47 | NEGMIR     |   291,732 |  6.06 |         1,326 |   583,470 | enough      |     73.50 | ever_enough              |    203.83 |
| **COM~ever_greater**                |    3.09 |   0.30 |    0.49 |   0.99 |    186 |        10,870 |    187 | ever         |       92.50 |            2.09 | COMPLEMENT | 3,173,552 |  6.78 |         6,949 | 6,347,364 | greater     |     93.50 | ever_greater             |    246.80 |
| **NEGmir~necessarily_wrong**        |    3.05 |   0.29 |    0.49 |   0.99 |    211 |           992 |    214 | necessarily  |      104.00 |            1.78 | NEGMIR     |   291,732 |  7.16 |         8,506 |   583,470 | wrong       |    107.00 | necessarily_wrong        |    265.18 |
| **NEGmir~ever_certain**             |    3.04 |   0.30 |    0.50 |   1.00 |    143 |         4,786 |    143 | ever         |       71.50 |            2.46 | NEGMIR     |   291,732 |  5.98 |         1,276 |   583,470 | certain     |     71.50 | ever_certain             |    198.28 |
| **NEGany~any_different**            |    3.03 |   0.28 |    0.44 |   0.94 |    902 |        16,238 |    957 | any          |      423.50 |            1.21 | NEGATED    | 3,173,660 | 14.10 |        80,643 | 6,347,364 | different   |    478.50 | any_different            |    905.82 |
| **NEGany~terribly_popular**         |    2.99 |   0.30 |    0.50 |   1.00 |    149 |        19,802 |    149 | terribly     |       74.50 |            2.48 | NEGATED    | 3,173,660 |  6.10 |        51,120 | 6,347,364 | popular     |     74.50 | terribly_popular         |    206.56 |
| **NEGany~remotely_close**           |    2.98 |   0.28 |    0.45 |   0.95 |    694 |         6,161 |    733 | remotely     |      327.50 |            1.25 | NEGATED    | 3,173,660 | 12.43 |        46,485 | 6,347,364 | close       |    366.50 | remotely_close           |    711.52 |
| **NEGany~terribly_unusual**         |    2.96 |   0.30 |    0.50 |   1.00 |    146 |        19,802 |    146 | terribly     |       73.00 |            2.47 | NEGATED    | 3,173,660 |  6.04 |         7,412 | 6,347,364 | unusual     |     73.00 | terribly_unusual         |    202.40 |
| **COM~ever_larger**                 |    2.88 |   0.30 |    0.50 |   1.00 |    139 |        10,870 |    139 | ever         |       69.50 |            2.45 | COMPLEMENT | 3,173,552 |  5.90 |         7,453 | 6,347,364 | larger      |     69.50 | ever_larger              |    192.71 |
| **NEGany~immediately_sure**         |    2.87 |   0.30 |    0.50 |   1.00 |    138 |        58,040 |    138 | immediately  |       69.00 |            2.44 | NEGATED    | 3,173,660 |  5.87 |       134,139 | 6,347,364 | sure        |     69.00 | immediately_sure         |    191.31 |
| **NEGany~immediately_successful**   |    2.87 |   0.29 |    0.47 |   0.97 |    290 |        58,040 |    299 | immediately  |      140.50 |            1.49 | NEGATED    | 3,173,660 |  8.25 |        31,460 | 6,347,364 | successful  |    149.50 | immediately_successful   |    333.73 |
| **COM~particularly_acute**          |    2.84 |   0.30 |    0.50 |   1.00 |    135 |        76,162 |    135 | particularly |       67.50 |            2.43 | COMPLEMENT | 3,173,552 |  5.81 |         1,038 | 6,347,364 | acute       |     67.50 | particularly_acute       |    187.16 |
| **NEGany~terribly_comfortable**     |    2.77 |   0.30 |    0.50 |   1.00 |    129 |        19,802 |    129 | terribly     |       64.50 |            2.41 | NEGATED    | 3,173,660 |  5.68 |        23,908 | 6,347,364 | comfortable |     64.50 | terribly_comfortable     |    178.84 |
| **NEGmir~particularly_unusual**     |    2.72 |   0.29 |    0.48 |   0.98 |    170 |        10,029 |    173 | particularly |       83.50 |            1.69 | NEGMIR     |   291,732 |  6.40 |           933 |   583,470 | unusual     |     86.50 | particularly_unusual     |    209.60 |
| **NEGmir~that_great**               |    2.71 |   0.28 |    0.46 |   0.96 |    286 |         4,559 |    298 | that         |      137.00 |            1.36 | NEGMIR     |   291,732 |  8.10 |         2,123 |   583,470 | great       |    149.00 | that_great               |    312.65 |
| **NEGmir~ever_able**                |    2.71 |   0.30 |    0.49 |   0.99 |    136 |         4,786 |    137 | ever         |       67.50 |            1.96 | NEGMIR     |   291,732 |  5.79 |         1,891 |   583,470 | able        |     68.50 | ever_able                |    178.12 |
| **NEGmir~that_good**                |    2.65 |   0.27 |    0.44 |   0.94 |    447 |         4,559 |    476 | that         |      209.00 |            1.18 | NEGMIR     |   291,732 |  9.89 |        13,423 |   583,470 | good        |    238.00 | that_good                |    441.70 |
| **NEGany~terribly_bright**          |    2.61 |   0.30 |    0.50 |   1.00 |    117 |        19,802 |    117 | terribly     |       58.50 |            2.37 | NEGATED    | 3,173,660 |  5.41 |         8,623 | 6,347,364 | bright      |     58.50 | terribly_bright          |    162.20 |
| **NEGmir~remotely_close**           |    2.58 |   0.29 |    0.46 |   0.96 |    218 |         1,953 |    226 | remotely     |      105.00 |            1.41 | NEGMIR     |   291,732 |  7.11 |         4,831 |   583,470 | close       |    113.00 | remotely_close           |    244.21 |
| **COM~ever_higher**                 |    2.52 |   0.30 |    0.49 |   0.99 |    129 |        10,870 |    130 | ever         |       64.00 |            1.94 | COMPLEMENT | 3,173,552 |  5.64 |        12,992 | 6,347,364 | higher      |     65.00 | ever_higher              |    168.50 |
| **NEGmir~ever_wrong**               |    2.52 |   0.30 |    0.50 |   1.00 |    102 |         4,786 |    102 | ever         |       51.00 |            2.31 | NEGMIR     |   291,732 |  5.05 |         8,506 |   583,470 | wrong       |     51.00 | ever_wrong               |    141.42 |
| **NEGany~ever_good**                |    2.52 |   0.28 |    0.45 |   0.95 |    331 |        10,870 |    350 | ever         |      156.00 |            1.23 | NEGATED    | 3,173,660 |  8.57 |       201,244 | 6,347,364 | good        |    175.00 | ever_good                |    337.56 |
| **NEGany~immediately_reachable**    |    2.50 |   0.30 |    0.50 |   1.00 |    109 |        58,040 |    109 | immediately  |       54.50 |            2.34 | NEGATED    | 3,173,660 |  5.22 |           350 | 6,347,364 | reachable   |     54.50 | immediately_reachable    |    151.11 |
| **NEGany~particularly_athletic**    |    2.49 |   0.30 |    0.50 |   1.00 |    108 |        76,162 |    108 | particularly |       54.00 |            2.34 | NEGATED    | 3,173,660 |  5.20 |         1,772 | 6,347,364 | athletic    |     54.00 | particularly_athletic    |    149.72 |
| **NEGany~particularly_likeable**    |    2.46 |   0.30 |    0.50 |   1.00 |    106 |        76,162 |    106 | particularly |       53.00 |            2.33 | NEGATED    | 3,173,660 |  5.15 |           861 | 6,347,364 | likeable    |     53.00 | particularly_likeable    |    146.95 |
| **NEGany~terribly_common**          |    2.45 |   0.30 |    0.50 |   1.00 |    105 |        19,802 |    105 | terribly     |       52.50 |            2.32 | NEGATED    | 3,173,660 |  5.12 |        34,450 | 6,347,364 | common      |     52.50 | terribly_common          |    145.56 |
| **NEGmir~particularly_original**    |    2.33 |   0.30 |    0.50 |   1.00 |     90 |        10,029 |     90 | particularly |       45.00 |            2.26 | NEGMIR     |   291,732 |  4.74 |           715 |   583,470 | original    |     45.00 | particularly_original    |    124.78 |
| **NEGany~any_nicer**                |    2.30 |   0.30 |    0.50 |   1.00 |     96 |        16,238 |     96 | any          |       48.00 |            2.29 | NEGATED    | 3,173,660 |  4.90 |           642 | 6,347,364 | nicer       |     48.00 | any_nicer                |    133.09 |
| **NEGany~remotely_funny**           |    2.16 |   0.29 |    0.47 |   0.97 |    137 |         6,161 |    141 | remotely     |       66.50 |            1.49 | NEGATED    | 3,173,660 |  5.68 |        14,992 | 6,347,364 | funny       |     70.50 | remotely_funny           |    159.09 |
| **NEGany~inherently_evil**          |    2.12 |   0.26 |    0.41 |   0.91 |    358 |         8,614 |    392 | inherently   |      162.00 |            1.02 | NEGATED    | 3,173,660 |  8.56 |         3,171 | 6,347,364 | evil        |    196.00 | inherently_evil          |    312.23 |
| **NEGmir~that_big**                 |    2.08 |   0.29 |    0.47 |   0.97 |    113 |         4,559 |    116 | that         |       55.00 |            1.51 | NEGMIR     |   291,732 |  5.17 |         3,134 |   583,470 | big         |     58.00 | that_big                 |    132.98 |
| **NEGany~particularly_radical**     |    1.99 |   0.30 |    0.50 |   1.00 |     79 |        76,162 |     79 | particularly |       39.50 |            2.20 | NEGATED    | 3,173,660 |  4.44 |         2,637 | 6,347,364 | radical     |     39.50 | particularly_radical     |    109.52 |
| **NEGany~remotely_interested**      |    1.99 |   0.26 |    0.41 |   0.91 |    330 |         6,161 |    364 | remotely     |      148.00 |            0.98 | NEGATED    | 3,173,660 |  8.15 |        34,543 | 6,347,364 | interested  |    182.00 | remotely_interested      |    278.69 |
| **NEGany~any_smarter**              |    1.94 |   0.30 |    0.49 |   0.99 |     89 |        16,238 |     90 | any          |       44.00 |            1.78 | NEGATED    | 3,173,660 |  4.66 |           733 | 6,347,364 | smarter     |     45.00 | any_smarter              |    113.78 |
| **NEGmir~terribly_surprising**      |    1.85 |   0.30 |    0.50 |   1.00 |     67 |         2,204 |     67 | terribly     |       33.50 |            2.13 | NEGMIR     |   291,732 |  4.09 |         1,248 |   583,470 | surprising  |     33.50 | terribly_surprising      |     92.89 |
| **NEGany~ever_boring**              |    1.84 |   0.30 |    0.50 |   1.00 |     72 |        10,870 |     72 | ever         |       36.00 |            2.16 | NEGATED    | 3,173,660 |  4.24 |         3,840 | 6,347,364 | boring      |     36.00 | ever_boring              |     99.82 |
| **NEGmir~inherently_bad**           |    1.83 |   0.27 |    0.44 |   0.94 |    148 |         3,342 |    158 | inherently   |       69.00 |            1.15 | NEGMIR     |   291,732 |  5.67 |         4,790 |   583,470 | bad         |     79.00 | inherently_bad           |    144.52 |
| **NEGany~immediately_certain**      |    1.80 |   0.30 |    0.50 |   1.00 |     70 |        58,040 |     70 | immediately  |       35.00 |            2.15 | NEGATED    | 3,173,660 |  4.18 |        11,334 | 6,347,364 | certain     |     35.00 | immediately_certain      |     97.04 |
| **NEGmir~that_close**               |    1.67 |   0.30 |    0.50 |   1.00 |     60 |         4,559 |     60 | that         |       30.00 |            2.08 | NEGMIR     |   291,732 |  3.87 |         4,831 |   583,470 | close       |     30.00 | that_close               |     83.19 |
| **NEGmir~any_worse**                |    1.66 |   0.29 |    0.47 |   0.97 |     87 |         1,095 |     90 | any          |       42.00 |            1.40 | NEGMIR     |   291,732 |  4.50 |         2,007 |   583,470 | worse       |     45.00 | any_worse                |     98.47 |
| **NEGany~remotely_surprising**      |    1.66 |   0.30 |    0.49 |   0.99 |     75 |         6,161 |     76 | remotely     |       37.00 |            1.70 | NEGATED    | 3,173,660 |  4.27 |        18,776 | 6,347,364 | surprising  |     38.00 | remotely_surprising      |     94.71 |
| **NEGmir~terribly_new**             |    1.64 |   0.29 |    0.49 |   0.99 |     69 |         2,204 |     70 | terribly     |       34.00 |            1.67 | NEGMIR     |   291,732 |  4.09 |         4,300 |   583,470 | new         |     35.00 | terribly_new             |     86.57 |
| **NEGany~remotely_comparable**      |    1.62 |   0.28 |    0.44 |   0.94 |    118 |         6,161 |    125 | remotely     |       55.50 |            1.20 | NEGATED    | 3,173,660 |  5.11 |         2,401 | 6,347,364 | comparable  |     62.50 | remotely_comparable      |    119.34 |
| **NEGmir~ever_black**               |    1.56 |   0.30 |    0.50 |   1.00 |     56 |         4,786 |     56 | ever         |       28.00 |            2.05 | NEGMIR     |   291,732 |  3.74 |           646 |   583,470 | black       |     28.00 | ever_black               |     77.64 |
| **NEGmir~that_popular**             |    1.54 |   0.29 |    0.48 |   0.98 |     65 |         4,559 |     66 | that         |       32.00 |            1.64 | NEGMIR     |   291,732 |  3.97 |         2,841 |   583,470 | popular     |     33.00 | that_popular             |     81.14 |
| **NEGmir~particularly_novel**       |    1.50 |   0.30 |    0.50 |   1.00 |     54 |        10,029 |     54 | particularly |       27.00 |            2.04 | NEGMIR     |   291,732 |  3.67 |           179 |   583,470 | novel       |     27.00 | particularly_novel       |     74.87 |
| **NEGany~remotely_ready**           |    1.49 |   0.30 |    0.50 |   1.00 |     58 |         6,161 |     58 | remotely     |       29.00 |            2.07 | NEGATED    | 3,173,660 |  3.81 |        29,583 | 6,347,364 | ready       |     29.00 | remotely_ready           |     80.41 |
| **NEGany~any_sweeter**              |    1.49 |   0.30 |    0.50 |   1.00 |     58 |        16,238 |     58 | any          |       29.00 |            2.07 | NEGATED    | 3,173,660 |  3.81 |           388 | 6,347,364 | sweeter     |     29.00 | any_sweeter              |     80.41 |
| **NEGmir~particularly_religious**   |    1.47 |   0.30 |    0.50 |   1.00 |     53 |        10,029 |     53 | particularly |       26.50 |            2.03 | NEGMIR     |   291,732 |  3.64 |           337 |   583,470 | religious   |     26.50 | particularly_religious   |     73.48 |
| **NEGany~inherently_better**        |    1.46 |   0.26 |    0.41 |   0.91 |    144 |         8,614 |    158 | inherently   |       65.00 |            1.00 | NEGATED    | 3,173,660 |  5.42 |        50,827 | 6,347,364 | better      |     79.00 | inherently_better        |    124.46 |
| **NEGany~particularly_flashy**      |    1.46 |   0.30 |    0.50 |   1.00 |     57 |        76,162 |     57 | particularly |       28.50 |            2.06 | NEGATED    | 3,173,660 |  3.77 |         1,732 | 6,347,364 | flashy      |     28.50 | particularly_flashy      |     79.02 |
| **NEGany~inherently_good**          |    1.46 |   0.24 |    0.36 |   0.86 |    283 |         8,614 |    329 | inherently   |      118.50 |            0.79 | NEGATED    | 3,173,660 |  7.04 |       201,244 | 6,347,364 | good        |    164.50 | inherently_good          |    189.85 |
| **NEGmir~remotely_true**            |    1.43 |   0.29 |    0.48 |   0.98 |     61 |         1,953 |     62 | remotely     |       30.00 |            1.61 | NEGMIR     |   291,732 |  3.84 |         2,850 |   583,470 | true        |     31.00 | remotely_true            |     75.72 |
| **NEGany~remotely_similar**         |    1.39 |   0.25 |    0.40 |   0.90 |    152 |         6,161 |    169 | remotely     |       67.50 |            0.94 | NEGATED    | 3,173,660 |  5.48 |        11,088 | 6,347,364 | similar     |     84.50 | remotely_similar         |    123.97 |
| **NEGany~any_brighter**             |    1.37 |   0.29 |    0.48 |   0.98 |     63 |        16,238 |     64 | any          |       31.00 |            1.63 | NEGATED    | 3,173,660 |  3.91 |           640 | 6,347,364 | brighter    |     32.00 | any_brighter             |     78.42 |
| **NEGmir~necessarily_bad**          |    1.37 |   0.30 |    0.50 |   1.00 |     50 |           992 |     50 | necessarily  |       25.00 |            2.00 | NEGMIR     |   291,732 |  3.54 |         4,790 |   583,470 | bad         |     25.00 | necessarily_bad          |     69.32 |
| **NEGmir~immediately_available**    |    1.34 |   0.25 |    0.38 |   0.88 |    162 |           564 |    184 | immediately  |       70.00 |            0.86 | NEGMIR     |   291,732 |  5.50 |         3,079 |   583,470 | available   |     92.00 | immediately_available    |    120.41 |
| **NEGmir~ever_right**               |    1.33 |   0.30 |    0.50 |   1.00 |     49 |         4,786 |     49 | ever         |       24.50 |            2.00 | NEGMIR     |   291,732 |  3.50 |         2,038 |   583,470 | right       |     24.50 | ever_right               |     67.93 |
| **NEGany~remotely_related**         |    1.33 |   0.25 |    0.40 |   0.90 |    146 |         6,161 |    163 | remotely     |       64.50 |            0.92 | NEGATED    | 3,173,660 |  5.34 |        14,260 | 6,347,364 | related     |     81.50 | remotely_related         |    116.95 |
| **COM~ever_deeper**                 |    1.31 |   0.29 |    0.48 |   0.98 |     61 |        10,870 |     62 | ever         |       30.00 |            1.61 | COMPLEMENT | 3,173,552 |  3.84 |         1,768 | 6,347,364 | deeper      |     31.00 | ever_deeper              |     75.72 |
| **NEGmir~any_different**            |    1.30 |   0.30 |    0.50 |   1.00 |     48 |         1,095 |     48 | any          |       24.00 |            1.99 | NEGMIR     |   291,732 |  3.46 |         8,644 |   583,470 | different   |     24.00 | any_different            |     66.55 |
| **NEGmir~terribly_interesting**     |    1.29 |   0.29 |    0.48 |   0.98 |     56 |         2,204 |     57 | terribly     |       27.50 |            1.58 | NEGMIR     |   291,732 |  3.67 |         3,863 |   583,470 | interesting |     28.50 | terribly_interesting     |     68.96 |
| **NEGmir~particularly_innovative**  |    1.26 |   0.30 |    0.50 |   1.00 |     47 |        10,029 |     47 | particularly |       23.50 |            1.98 | NEGMIR     |   291,732 |  3.43 |           675 |   583,470 | innovative  |     23.50 | particularly_innovative  |     65.16 |
| **NEGmir~that_interested**          |    1.26 |   0.29 |    0.47 |   0.97 |     62 |         4,559 |     64 | that         |       30.00 |            1.40 | NEGMIR     |   291,732 |  3.81 |         2,877 |   583,470 | interested  |     32.00 | that_interested          |     70.93 |
| **NEGany~inherently_illegal**       |    1.26 |   0.29 |    0.48 |   0.98 |     59 |         8,614 |     60 | inherently   |       29.00 |            1.60 | NEGATED    | 3,173,660 |  3.78 |         3,580 | 6,347,364 | illegal     |     30.00 | inherently_illegal       |     73.01 |
| **NEGmir~any_easier**               |    1.23 |   0.29 |    0.47 |   0.97 |     61 |         1,095 |     63 | any          |       29.50 |            1.39 | NEGMIR     |   291,732 |  3.78 |           681 |   583,470 | easier      |     31.50 | any_easier               |     69.61 |
| **NEGmir~terribly_original**        |    1.19 |   0.30 |    0.50 |   1.00 |     45 |         2,204 |     45 | terribly     |       22.50 |            1.96 | NEGMIR     |   291,732 |  3.35 |           715 |   583,470 | original    |     22.50 | terribly_original        |     62.39 |
| **NEGmir~that_difficult**           |    1.16 |   0.29 |    0.48 |   0.98 |     52 |         4,559 |     53 | that         |       25.50 |            1.54 | NEGMIR     |   291,732 |  3.54 |         4,854 |   583,470 | difficult   |     26.50 | that_difficult           |     63.56 |
| **NEGmir~exactly_clear**            |    1.16 |   0.29 |    0.48 |   0.98 |     52 |           869 |     53 | exactly      |       25.50 |            1.54 | NEGMIR     |   291,732 |  3.54 |         3,321 |   583,470 | clear       |     26.50 | exactly_clear            |     63.56 |
| **NEGmir~particularly_comfortable** |    1.15 |   0.30 |    0.50 |   1.00 |     44 |        10,029 |     44 | particularly |       22.00 |            1.95 | NEGMIR     |   291,732 |  3.32 |         1,888 |   583,470 | comfortable |     22.00 | particularly_comfortable |     61.00 |
| **NEGmir~remotely_comparable**      |    1.15 |   0.30 |    0.50 |   1.00 |     44 |         1,953 |     44 | remotely     |       22.00 |            1.95 | NEGMIR     |   291,732 |  3.32 |           158 |   583,470 | comparable  |     22.00 | remotely_comparable      |     61.00 |
| **NEGany~remotely_enough**          |    1.13 |   0.30 |    0.50 |   1.00 |     47 |         6,161 |     47 | remotely     |       23.50 |            1.98 | NEGATED    | 3,173,660 |  3.43 |        27,603 | 6,347,364 | enough      |     23.50 | remotely_enough          |     65.16 |
| **POS~terribly_wrong**              |    1.06 |   0.20 |    0.30 |   0.80 |    319 |         2,204 |    401 | terribly     |      118.50 |            0.59 | POSMIR     |   291,729 |  6.63 |         8,506 |   583,470 | wrong       |    200.50 | terribly_wrong           |    149.75 |
| **COM~ever_mindful**                |    1.04 |   0.29 |    0.48 |   0.98 |     52 |        10,870 |     53 | ever         |       25.50 |            1.54 | COMPLEMENT | 3,173,552 |  3.54 |           784 | 6,347,364 | mindful     |     26.50 | ever_mindful             |     63.56 |
| **NEGmir~that_happy**               |    1.03 |   0.30 |    0.50 |   1.00 |     41 |         4,559 |     41 | that         |       20.50 |            1.92 | NEGMIR     |   291,732 |  3.20 |         5,463 |   583,470 | happy       |     20.50 | that_happy               |     56.84 |



In [13]:
# %%

nb_show_table(NEG_bigrams_sample.l1.value_counts().to_frame('subtotal in selected bigrams'))


|                |   `subtotal in selected bigrams` |
|:---------------|---------------------------------:|
| **NEGATED**    |                               99 |
| **NEGMIR**     |                               48 |
| **COMPLEMENT** |                                7 |
| **POSMIR**     |                                1 |



|                |   `subtotal in selected bigrams` |
|:---------------|---------------------------------:|
| **NEGATED**    |                               99 |
| **NEGMIR**     |                               48 |
| **COMPLEMENT** |                                7 |
| **POSMIR**     |                                1 |

In [16]:
nb_show_table(NEG_bigrams_sample.filter(like='O', axis=0))


|                            |   `LRC` |   `MI` |   `dP1` |   `P1` |   `f` |   `adv_total` |   `f2` | `adv`        |   `unexp_f` |   `odds_r_disc` | `l1`       |      `f1` |   `t` |   `adj_total` |       `N` | `adj`   |   `exp_f` | `l2`               |   `G2` |
|:---------------------------|--------:|-------:|--------:|-------:|------:|--------------:|-------:|:-------------|------------:|----------------:|:-----------|----------:|------:|--------------:|----------:|:--------|----------:|:-------------------|-------:|
| **COM~ever_closer**        |    3.52 |   0.30 |    0.49 |   0.99 |   279 |        10,870 |    281 | ever         |      138.51 |            2.05 | COMPLEMENT | 3,173,552 |  8.29 |         3,686 | 6,347,364 | closer  |    140.49 | ever_closer        | 365.82 |
| **COM~ever_greater**       |    3.09 |   0.30 |    0.49 |   0.99 |   186 |        10,870 |    187 | ever         |       92.50 |            2.09 | COMPLEMENT | 3,173,552 |  6.78 |         6,949 | 6,347,364 | gre

COM associations of selected bigrams

|                            |   `LRC` |   `MI` |   `dP1` |   `P1` |   `f` |   `adv_total` |   `f2` | `adv`        |   `unexp_f` |   `odds_r_disc` | `l1`       |      `f1` |   `t` |   `adj_total` |       `N` | `adj`   |   `exp_f` | `l2`               |   `G2` |
|:---------------------------|--------:|-------:|--------:|-------:|------:|--------------:|-------:|:-------------|------------:|----------------:|:-----------|----------:|------:|--------------:|----------:|:--------|----------:|:-------------------|-------:|
| **COM~ever_closer**        |    3.52 |   0.30 |    0.49 |   0.99 |   279 |        10,870 |    281 | ever         |      138.51 |            2.05 | COMPLEMENT | 3,173,552 |  8.29 |         3,686 | 6,347,364 | closer  |    140.49 | ever_closer        | 365.82 |
| **COM~ever_greater**       |    3.09 |   0.30 |    0.49 |   0.99 |   186 |        10,870 |    187 | ever         |       92.50 |            2.09 | COMPLEMENT | 3,173,552 |  6.78 |         6,949 | 6,347,364 | greater |     93.50 | ever_greater       | 246.80 |
| **COM~ever_larger**        |    2.88 |   0.30 |    0.50 |   1.00 |   139 |        10,870 |    139 | ever         |       69.50 |            2.45 | COMPLEMENT | 3,173,552 |  5.90 |         7,453 | 6,347,364 | larger  |     69.50 | ever_larger        | 192.71 |
| **COM~particularly_acute** |    2.84 |   0.30 |    0.50 |   1.00 |   135 |        76,162 |    135 | particularly |       67.50 |            2.43 | COMPLEMENT | 3,173,552 |  5.81 |         1,038 | 6,347,364 | acute   |     67.50 | particularly_acute | 187.16 |
| **COM~ever_higher**        |    2.52 |   0.30 |    0.49 |   0.99 |   129 |        10,870 |    130 | ever         |       64.00 |            1.94 | COMPLEMENT | 3,173,552 |  5.64 |        12,992 | 6,347,364 | higher  |     65.00 | ever_higher        | 168.50 |
| **COM~ever_deeper**        |    1.31 |   0.29 |    0.48 |   0.98 |    61 |        10,870 |     62 | ever         |       30.00 |            1.61 | COMPLEMENT | 3,173,552 |  3.84 |         1,768 | 6,347,364 | deeper  |     31.00 | ever_deeper        |  75.72 |
| **POS~terribly_wrong**     |    1.06 |   0.20 |    0.30 |   0.80 |   319 |         2,204 |    401 | terribly     |      118.50 |            0.59 | POSMIR     |   291,729 |  6.63 |         8,506 |   583,470 | wrong   |    200.50 | terribly_wrong     | 149.75 |
| **COM~ever_mindful**       |    1.04 |   0.29 |    0.48 |   0.98 |    52 |        10,870 |     53 | ever         |       25.50 |            1.54 | COMPLEMENT | 3,173,552 |  3.54 |           784 | 6,347,364 | mindful |     26.50 | ever_mindful       |  63.56 |

